In [791]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import folium
from folium.plugins import MarkerCluster
from datetime import datetime
import seaborn as sns
from collections import defaultdict
from helpers import parse_feature_metadata
import osmnx as ox
import geopandas as gpd


Please set this constant to false if you wish the cells not to plot graphs

In [792]:
PLOT_GRAPHS = False

Local paths should be correct if repository was cloned

In [793]:
NEPAL_PATH = r"data\raw_updated_names\nepal_new_feature_names.xlsx"
SENEGAL_PATH = r"data\raw_updated_names\senegal_new_feature_names.xlsx"

nepal_df = pd.read_excel(NEPAL_PATH, index_col=0)
senegal_df = pd.read_excel(SENEGAL_PATH, index_col=0)

## 1. Initial Parsing of the Data
This phase includes removal of features that we from the start knew we would not use or would not be beneficial, and also coerces the remaining features's values to be their declared types

In [794]:
NEPAL_CAT_COL_NAMES = {'Q11': 'Marital_Status',
 'Q53': 'What_type_of_crop_is_grown_on_this_plot',
 'Q54': 'For_vegetables_what_is_your_source_of_seeds',
 'Q56': 'For_vegetables_do_you_use_seedlings',
 'Q58': 'fertilizer_on_this_plot',
 'Q63': 'What_is_the_main_use_of_produce_from_holding',
 'Q64': 'Do_you_use_machinery_or_and_equipment_on_the_plot',
 'Q65': 'Do_you_do_any_of_the_following',
 'Q67': 'What_do_you_use_soil_analysis_for',
 'Q68': 'How_do_you_conduct_soil_analysis',
 'Q69': 'What_is_correct_for_you',
 'Q71': 'in_the_past_12_months_from_who_did_you_receive_info_on_agriculture',
 'Q73': 'Did_you_receive_anything_from_these_organizations',
 'Q74': 'How_do_you_decide_to_plow',
 'Q75': 'How_do_you_decide_to_begin_sowing',
 'Q76': 'What_type_of_irrigation_do_you_use',
 'Q100': 'Caste',
 'Q105': 'main_sources_of_income',
 'Q111': 'Generally_speaking_would_you_say_that_most_people_can_be_trusted',
 'Q112': 'I_am_much_better_than_most_farmers_here',
 'Q113': 'dislike_not_knowing_what_is_going_to_happen'}

SENEGAL_CAT_COL_NAMES = {'Q63': 'CROP',
 'Q64': 'Seed_source',
 'Q65': 'Variety',
 'Q66': 'Seedlings',
 'Q67': 'Fertilizer',
 'Q72': 'Sold_VEG',
 'Q73': 'Machinery',
 'Q74': 'Practice',
 'Q77': 'Info_source',
 'Q79': 'Did_you_receive_anything_from_the_specified_organizations',
 'Q80': 'Plow_weather',
 'Q81': 'Sow',
 'Q82': 'Irrigation',
 'Q88': 'family_main_sources_income',
 'Q89': 'education_level'}

def fix_column_values(series: pd.Series, meta: dict) -> pd.Series:
    """
    Given a pandas Series and its parsed metadata,
    coerce its values to the right type:
      - continuous, ordinal → numeric (floats or ints; bad → NaN)
      - nominal -> categorical
      - binary → 0/1 integers   (anything non-zero → 1, missing -> NaN)
      - time   → pandas datetime (bad → NaT)
    """
    ftype = meta["type"]
    fname = meta["name"].lower()

    if ftype in ("continuous", "ordinal"):
        # numeric codes or measurements
        return pd.to_numeric(series, errors="coerce")
    if ftype == "nominal":
        return series.astype("category")
    if "binary" in ftype:
        # strings that are not '0': → 1, otherwise __> 0, floats/ints: non-zero → 1, zero or NaN → 0
        def to_binary(x):
            if pd.isna(x):
                return pd.NA
            if isinstance(x, str):
                return x != '0'
            try:
                return 1 if float(x) != 0 else 0
            except:
                return 1
        return series.map(to_binary).astype("Int64")

    if ftype == "time":
        # Force everything to str so we have uniform input
        s = series.astype(str)
        # 0) special: End_Date holds a full datetime → extract only the time
        if "end_date" in fname:
            # parses strings like "4/27/2018 9:47:17 AM" into Timestamps
            dt = pd.to_datetime(s, errors="coerce")
            # grab Python datetime.time
            return dt.dt.time
        # 1) true date column → datetime64
        if "date" in fname:
            return pd.to_datetime(s, format="%Y-%m-%d", errors="coerce")

        # 2) survey‐length durations → timedelta64
        if "length" in fname:
            # strings like "00:21:14" → Timedelta
            return pd.to_timedelta(s, errors="coerce")

        # 3) the two pure clock‐time columns → Python time
        #    strings like "11:27:52" → Timestamp → .time()
        parsed = pd.to_datetime(s, format="%H:%M:%S", errors="coerce")
        return parsed.dt.time

    # otherwise leave it alone
    return series


def clean_survey_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    """
    Returns a copy of df in which:
      - all columns named Q<digits>__… have been coerced to their declared types
      - all other columns are left untouched (you can drop them later)
    """
    df = df.copy()
    for col in df.columns:
        meta = parse_feature_metadata(col)
        if meta is None:
            continue
        df[col] = fix_column_values(df[col], meta)
    return df


def drop_non_relevant_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    Remove everything except Q<digits> columns:
    """
    keep = [col for col in df.columns if parse_feature_metadata(col) is not None]
    return df[keep]

import pandas as pd
from collections import defaultdict

def combine_dummy_columns(orig_df: pd.DataFrame, names_dict: dict) -> pd.DataFrame:
    """
    Combines your binary dummy columns into single categorical columns,
    but now *preserves* the option labels.

    For each (qid, type):
      - If there's exactly one dummy → just rename it to end in `-1`.
      - If there are many and it's one-hot (max 1 per row) → new column takes the option LABEL, or None.
      - Otherwise (multi-select) → new column is a list of the LABELS selected (possibly empty).
    """
    df = orig_df.copy()

    # 1) Gather metadata & group columns by (qid, type)
    meta_map   = {}               # col_name → {qid, qname, var_type, dummy_idx, label}
    buckets    = defaultdict(list)

    for col in df.columns:
        if "binary" not in col:
            continue
        m = parse_feature_metadata(col)
        qid        = m["qid"]
        qname      = m["name"]
        var_type   = m["type"]
        dummy_idx  = m["dummy"]
        
        subnames = qname.split(':')
        label      = qname

        meta_map[col] = {
            "qid":       qid,
            "qname":     qname,
            "var_type":  var_type,
            "dummy_idx": dummy_idx,
            "label":     label
        }
        buckets[(qid, var_type)].append(col)

    to_drop  = []
    new_cols = {}

    # 2) Process each question group
    for (qid, var_type), cols in buckets.items():
        # sort by dummy_idx so labels stay in the right order
        cols_sorted   = sorted(cols, key=lambda c: meta_map[c]["dummy_idx"])
        labels_sorted = [meta_map[c]["label"] for c in cols_sorted]
        arr           = df[cols_sorted].fillna(0).astype(int).values

        # A) Single-dummy → just rename it to ...-1
        if len(cols_sorted) == 1:
            old = cols_sorted[0]
            new = f"{qid}__{meta_map[old]['qname']}__binary__1"
            df = df.rename(columns={old: new})
            continue
        

        new_type = "nominal" if var_type.endswith("nominal") else "ordinal"
        qname = names_dict[qid]
        new_name = f"{qid}__{qname}__{new_type}"
        

        # B) One-hot? (no row has more than one “1”)
        if arr.sum(axis=1).max() <= 1:
            cat = []
            for row in arr:
                if row.sum() == 0:
                    cat.append(None)
                else:
                    cat.append(labels_sorted[row.argmax()])
            
            new_cols[new_name] = cat

        # C) Multi-select
        else:
            multi = []
            for row in arr:
                # collect all the labels whose dummy==1
                sel = [labels_sorted[i] for i, v in enumerate(row) if v == 1]
                multi.append(tuple(sel))
            new_cols[new_name] = multi

        to_drop.extend(cols_sorted)

    # 3) Drop old dummies & add the new columns
    if to_drop:
        df = df.drop(columns=to_drop)
        df = pd.concat([df, pd.DataFrame(new_cols, index=df.index)], axis=1)

    return df, new_cols



In [795]:
# dropping non Q#... columns
nepal_df = drop_non_relevant_columns(nepal_df)
senegal_df = drop_non_relevant_columns(senegal_df)

# initial clean
nepal_df = clean_survey_dataframe(nepal_df)
senegal_df = clean_survey_dataframe(senegal_df)

# combine and remove dummy columns
nepal_df, npl_new_cols = combine_dummy_columns(nepal_df, NEPAL_CAT_COL_NAMES)
senegal_df, sng_new_cols = combine_dummy_columns(senegal_df, SENEGAL_CAT_COL_NAMES)

## Create Productivity Metric Columns (Targets)

In [796]:
BIG_TO_SQM = 6772.63
HEC_TO_SQM = 10_000
XOF_TO_USD = 0.001871
NP_TO_USD = 0.008851

NPL_YEARLY_AGR_INCOME_COL = "Q108__What_is_your_households_yearly_income_from_agriculture_NPR__continuous"
SNGL_YEARLY_AGR_INCOME_COL = 'Q90__Yearly_income_agriculture_XOF__continuous'
NPL_OWND_CULTVTD_LAND_COL = "Q50__How_much_land_that_is_yours_do_you_cultivate_bigha__continuous"
SNGL_OWND_CULTVTD_LAND_COL = "Q60__Land_owned_cultivated_ha__continuous"
NPL_LEAS_CULTVTD_LAND_COL = "Q51__How_much_land_that_is_rented_or_leased_do_you_cultivate_bigha__continuous"
SNGL_LEAS_CULTVTD_LAND_COL = "Q61__Land_rented_cultivated_ha__continuous"
NPL_VEG_HARVEST_PER_YEAR_COL = "Q62__How_much_VEGETABLES_do_you_harvest_per_year_from_this_plot_kilograms__continuous"
SNGL_VEG_HARVEST_PER_YEAR_COL = "Q71__VEG_harvest_per_year_kg__continuous"
NPL_SELF_REPORTED_FARM_LEVEL_COL = "Q112__Generally_speaking_how_would_you_define_your_farming__ordinal"
SNGL_SELF_REPORTED_FARM_LEVEL_COL = "Q94__Farming_level_relative__ordinal"
NPL_VEG_LAND = "Q52__On_how_much_land_do_you_grow_vegetables_bigha__continuous"
SNGL_VEG_LAND = "Q62__Land_grow_vegetables_ha__continuous"

nepal_df['Q0__target1_yearly_income_from_agr_per_land_SQM__continuous'] = (NP_TO_USD * nepal_df[NPL_YEARLY_AGR_INCOME_COL]) / (BIG_TO_SQM * (nepal_df[NPL_OWND_CULTVTD_LAND_COL] + nepal_df[NPL_LEAS_CULTVTD_LAND_COL]))
senegal_df['Q0__target1_yearly_income_from_agr_per_land_SQM__continuous'] = (XOF_TO_USD * senegal_df[SNGL_YEARLY_AGR_INCOME_COL]) / (HEC_TO_SQM * (senegal_df[SNGL_OWND_CULTVTD_LAND_COL] + senegal_df[SNGL_LEAS_CULTVTD_LAND_COL]))

nepal_df['Q0__target2_yearly_income_from_agr_USD__continuous'] = NP_TO_USD * nepal_df[NPL_YEARLY_AGR_INCOME_COL]
senegal_df['Q0__target2_yearly_income_from_agr_USD__continuous'] = XOF_TO_USD * senegal_df[SNGL_YEARLY_AGR_INCOME_COL]

nepal_df['Q0__target3_veg_per_area__continuous'] = nepal_df[NPL_VEG_HARVEST_PER_YEAR_COL] / (BIG_TO_SQM * nepal_df[NPL_VEG_LAND])
senegal_df['Q0__target3_veg_per_area__continuous'] = senegal_df[SNGL_VEG_HARVEST_PER_YEAR_COL] / (HEC_TO_SQM * senegal_df[SNGL_VEG_LAND])

nepal_df.rename(columns={NPL_SELF_REPORTED_FARM_LEVEL_COL: 'Q0__target4_self_farming_perception__ordinal'}, inplace=True)
senegal_df.rename(columns={SNGL_SELF_REPORTED_FARM_LEVEL_COL: 'Q0__target4_self_farming_perception__ordinal'}, inplace=True)

# defragment
nepal_df = nepal_df.copy()
senegal_df = senegal_df.copy()

target_cols = ['Q0__target1_yearly_income_from_agr_per_land_SQM__continuous',
               'Q0__target2_yearly_income_from_agr_USD__continuous',
               'Q0__target3_veg_per_area__continuous',
               'Q0__target4_self_farming_perception__ordinal']

# Univariate Analysis

## Analyze Datetime columns

Create mew datetime column

In [797]:


nepal_df['Q2__SurveyedDate__time'] = pd.to_datetime(nepal_df['Q2__SurveyedDate__time'], errors='coerce')

def create_datetime_col(df):
    # ensure your date column is datetime64
    df['Q2__SurveyedDate__time'] = pd.to_datetime(
        df['Q2__SurveyedDate__time'], format="%Y-%m-%d", errors='coerce'
    )

    # 1) build an all‐string “HH:MM:SS” series
    time_str = df['Q2__SurveyedTime__time'].astype(str).str.extract(r'(\d{2}:\d{2}:\d{2})')[0]

    # 2) combine
    dt_strings = df['Q2__SurveyedDate__time'].dt.strftime('%Y-%m-%d') + ' ' + time_str

    # 3) parse into real timestamps
    df['Q2__Surveyed_Date_Time__time'] = pd.to_datetime(dt_strings, errors='coerce')

    print(df['Q2__Surveyed_Date_Time__time'].dtype)  # should be datetime64[ns]
    
create_datetime_col(nepal_df)
create_datetime_col(senegal_df)

datetime64[ns]
datetime64[ns]


Removed old one since already have DateTime column

In [798]:

nepal_df = nepal_df.drop(['Q2__SurveyedDate__time', 'Q2__SurveyedTime__time'], axis=1)
senegal_df = senegal_df.drop(['Q2__SurveyedDate__time', 'Q2__SurveyedTime__time'], axis=1)

Define function to analyze new datetime columns

In [799]:

def analyze_datetime(df, col):
    """
    Perform univariate analysis on a datetime64 column:
      - prints basic stats (min, max, missing)
      - plots counts per day, month, weekday, and hour-of-day
    """
    # Basic info
    series = df[col]
    if not series.dtype.name.startswith('datetime'):
        raise ValueError(f"Column {col!r} is not datetime dtype.")
    
    print(f"\n--- ANALYSIS OF {col.upper()} ---")
    print(f"Type: {series.dtype}")
    print(f"Missing: {series.isna().sum()} / {len(series)}")
    print(f"Range: {series.min()} → {series.max()}")
    
    # Extract time features
    df_temp = df.dropna(subset=[col]).copy()
    df_temp['date']    = df_temp[col].dt.date
    df_temp['month']   = df_temp[col].dt.to_period('M').astype(str)
    df_temp['weekday'] = df_temp[col].dt.day_name()
    df_temp['hour']    = df_temp[col].dt.hour
    
    # 1. Counts per day
    daily = df_temp.groupby('date').size()
    plt.figure()
    daily.plot(kind='bar')
    plt.title(f"Number of Records per Day ({col})")
    plt.xlabel("Date")
    plt.ylabel("Count")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    
    # 2. Counts by month
    monthly = df_temp.groupby('month').size().sort_index()
    plt.figure()
    monthly.plot(kind='bar')
    plt.title(f"Counts by Month ({col})")
    plt.xlabel("Month")
    plt.ylabel("Count")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    
    # 3. Counts by weekday
    # ensure Mon–Sun order
    weekdays = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
    wd = df_temp['weekday'].value_counts().reindex(weekdays)
    plt.figure()
    wd.plot(kind='bar')
    plt.title(f"Counts by Weekday ({col})")
    plt.xlabel("Weekday")
    plt.ylabel("Count")
    plt.tight_layout()
    plt.show()
    
    # 4. Distribution of hour of day
    plt.figure()
    sns.histplot(df_temp['hour'], bins=24, discrete=True, kde=False)
    plt.title(f"Survey Time of Day Distribution ({col})")
    plt.xlabel("Hour of Day")
    plt.ylabel("Count")
    plt.xticks(range(0,24))
    plt.tight_layout()
    plt.show()

    

Analyze new datetime columns

In [800]:
if PLOT_GRAPHS: 
    print("NEPAL DATA")
    analyze_datetime(nepal_df, 'Q2__Surveyed_Date_Time__time')
    print("SENEGAL DATA")
    analyze_datetime(senegal_df, 'Q2__Surveyed_Date_Time__time')

## Analyze Survey Length

Add Survey_Length column

In [801]:

# turn time → "HH:MM:SS" strings → Timedelta
nepal_df['Q0__SurveyLengthTime__time'] = (
    nepal_df['Q0__SurveyLengthTime__time']
      .astype(str)                # datetime.time → "HH:MM:SS"
      .pipe(pd.to_timedelta)      # parse into timedelta64[ns]
)

# first, make sure both your start‐ and end‐times are strings
senegal_df['Q2__Surveyed_Date_Time__time'] = senegal_df['Q2__Surveyed_Date_Time__time'].astype(str)
senegal_df['Q0__Surveyed_End_Date__time']   = senegal_df['Q0__Surveyed_End_Date__time'].astype(str)

# 1) Parse your “start” column as a real timestamp
senegal_df['start_dt'] = pd.to_datetime(
    senegal_df['Q2__Surveyed_Date_Time__time'],
    format='%Y-%m-%d %H:%M:%S'
)

# 2) Turn your “end time” (just HH:MM:SS) into a Timedelta
senegal_df['end_td'] = pd.to_timedelta(
    senegal_df['Q0__Surveyed_End_Date__time'].astype(str)
)

# 3) Build a full “end” timestamp by taking the date from start_dt and adding the time delta
senegal_df['end_dt'] = (
    senegal_df['start_dt'].dt.normalize()  # midnight of the start date
  + senegal_df['end_td']
)

senegal_df['Q0__SurveyLengthTime__time'] = senegal_df['end_dt'] - senegal_df['start_dt']

senegal_df.drop(columns=['start_dt', 'end_td', 'end_dt'], inplace=True)

Function to analyze Survey_Length column

In [802]:
from matplotlib.ticker import FuncFormatter

def analyze_survey_length(df, column='Q0__SurveyLengthTime__time', bins=30):
    """
    Analyze the distribution of survey lengths in a DataFrame.

    Parameters
    ----------
    df : pandas.DataFrame
        The DataFrame containing the survey length column.
    column : str, default 'Survey_Length'
        Name of the timedelta column to analyze.
    bins : int, default 30
        Number of histogram bins.

    Outputs
    -------
    - Histogram of survey lengths (HH:MM:SS) distribution.
    - Printed mean and standard deviation of survey lengths.
    - Returns a tuple (mean_timedelta, std_timedelta).
    """
    # Ensure the column is timedelta64[ns]
    if not pd.api.types.is_timedelta64_dtype(df[column]):
        raise TypeError(f"Column '{column}' must be timedelta64[ns] dtype")

    # Compute statistics
    mean_td = df[column].mean()
    std_td = df[column].std()

    # Convert to total seconds for plotting
    secs = df[column].dt.total_seconds()

    # Plot histogram
    fig, ax = plt.subplots()
    ax.hist(secs, bins=bins)
    ax.set_title("Distribution of Survey Lengths")
    ax.set_xlabel("Survey Length (HH:MM:SS)")
    ax.set_ylabel("Frequency")

    # Formatter to convert seconds back to HH:MM:SS
    def sec_to_hhmmss(x, pos):
        h = int(x // 3600)
        m = int((x % 3600) // 60)
        s = int(x % 60)
        return f"{h:02d}:{m:02d}:{s:02d}"

    ax.xaxis.set_major_formatter(FuncFormatter(sec_to_hhmmss))
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

    # Print mean and std in HH:MM:SS
    mean_str = str(mean_td).split()[-1]
    std_str  = str(std_td).split()[-1]
    print(f"Mean survey length: {mean_str}")
    print(f"Std dev survey length: {std_str}")

    return mean_td, std_td

In [803]:
if PLOT_GRAPHS:
    print("NEPAL DATA")
    analyze_survey_length(nepal_df)
    print("SENEGAL DATA")
    analyze_survey_length(senegal_df)

Discovered data-entry glitch extreme value at one Senegalese survey

In [804]:
bad = senegal_df[ senegal_df['Q0__SurveyLengthTime__time'] < pd.Timedelta(0) ]
print(bad[['Q2__Surveyed_Date_Time__time','Q0__Surveyed_End_Date__time','Q0__SurveyLengthTime__time']])

    Q2__Surveyed_Date_Time__time Q0__Surveyed_End_Date__time  \
328          2018-04-27 14:18:09                    00:11:33   

    Q0__SurveyLengthTime__time  
328          -1 days +09:53:24  


Filling in this survey with median survey length

In [805]:
median_td = senegal_df.loc[ senegal_df['Q0__SurveyLengthTime__time'] >= pd.Timedelta(0), 'Q0__SurveyLengthTime__time' ].median()
mask = senegal_df['Q0__SurveyLengthTime__time'] < pd.Timedelta(0)
print(f"Found {mask.sum()} negative Survey_Length(s), filling with median = {median_td}")
senegal_df.loc[mask, 'Q0__SurveyLengthTime__time'] = median_td

Found 1 negative Survey_Length(s), filling with median = 0 days 00:31:58


Redefine survey analysis function by adding standard deviation markers

In [806]:
from matplotlib.ticker import FuncFormatter

def analyze_survey_length(df, column='Q0__SurveyLengthTime__time', bins=30, max_std=3):
    """
    Analyze the distribution of survey lengths in a DataFrame,
    and overlay vertical lines at mean ± n * std for n = 1..max_std.

    Parameters
    ----------
    df : pandas.DataFrame
        The DataFrame containing the survey length column.
    column : str, default 'Survey_Length'
        Name of the timedelta column to analyze.
    bins : int, default 30
        Number of histogram bins.
    max_std : int, default 3
        How many standard-deviation multiples to draw lines for.

    Outputs
    -------
    - Histogram of survey lengths (HH:MM:SS) distribution with σ-lines.
    - Printed mean and standard deviation of survey lengths.
    - Returns a tuple (mean_timedelta, std_timedelta).
    """
    # 1) Type check
    if not pd.api.types.is_timedelta64_dtype(df[column]):
        raise TypeError(f"Column '{column}' must be timedelta64[ns] dtype")

    # 2) Compute statistics
    mean_td = df[column].mean()
    std_td  = df[column].std()

    mean_sec = mean_td.total_seconds()
    std_sec  = std_td.total_seconds()

    # 3) Build histogram
    secs = df[column].dt.total_seconds()
    fig, ax = plt.subplots(figsize=(8,4))
    ax.hist(secs, bins=bins, edgecolor='black', alpha=0.7)
    ax.set_title("Distribution of Survey Lengths")
    ax.set_xlabel("Survey Length (HH:MM:SS)")
    ax.set_ylabel("Frequency")

    # Formatter: seconds → HH:MM:SS
    def sec_to_hhmmss(x, pos):
        h = int(x // 3600)
        m = int((x % 3600) // 60)
        s = int(x % 60)
        return f"{h:02d}:{m:02d}:{s:02d}"

    ax.xaxis.set_major_formatter(FuncFormatter(sec_to_hhmmss))
    plt.xticks(rotation=45)

    # 4) Overlay mean and std-bands
    #    use different linestyles/colors for clarity
    for n in range(0, max_std+1):
        # skip n=0 since that’s the mean line (draw it separately for legend clarity)
        if n == 0:
            ax.axvline(mean_sec, color='black', linestyle='-', linewidth=2,
                       label='Mean')
        else:
            for sign, label in [(+1, f'+{n}σ'), (-1, f'-{n}σ')]:
                x = mean_sec + sign * n * std_sec
                ax.axvline(x, linestyle='--' if n==1 else ':',
                           linewidth=1.5 if n==1 else 1,
                           label=label if sign>0 else None)
    # Only one -σ label needed, so we skip labeling the negative side

    ax.legend(loc='upper right')
    plt.tight_layout()
    plt.show()

    # 5) Print out
    mean_str = str(mean_td).split()[-1]
    std_str  = str(std_td).split()[-1]
    print(f"Mean survey length: {mean_str}")
    print(f"Std dev survey length: {std_str}")

    return mean_td, std_td

In [807]:
if PLOT_GRAPHS:
    print("SENEGAL DATA")
    analyze_survey_length(senegal_df)

Outliers defined as beyond 3 standard deviations, we swap outliers with median survey length

In [808]:
threshold = senegal_df['Q0__SurveyLengthTime__time'].mean() + 2*senegal_df['Q0__SurveyLengthTime__time'].std()
mask = senegal_df['Q0__SurveyLengthTime__time'] > threshold
print(f"Changed {mask.sum()} outlier survey lengths to median")
senegal_df.loc[mask, 'Q0__SurveyLengthTime__time'] = median_td

Changed 3 outlier survey lengths to median


Recheck survey length histogram

In [809]:
if PLOT_GRAPHS:
    print("SENEGAL DATA")
    analyze_survey_length(senegal_df)

## Analyze Locational Data

Since added Survey_Length column, took out others

In [810]:

nepal_df = nepal_df.drop(['Q0__SurveyedEndTime__time'], axis=1)
senegal_df = senegal_df.drop(['Q0__Surveyed_End_Date__time'], axis=1)

Showing missing columns in the Senegal dataset's Location_... columns

In [811]:
for column in ['Latitude', 'Longitude', 'Altitude', 'Accuracy']:
    print(f"missing values in Q1__Location_{column}: {senegal_df[f'Q1__Location_{column}__continuous'].isna().sum()}")
    print(f"missing values in {column}: {senegal_df[f'Q1__{column}__continuous'].isna().sum()}")


missing values in Q1__Location_Latitude: 335
missing values in Latitude: 0
missing values in Q1__Location_Longitude: 335
missing values in Longitude: 0
missing values in Q1__Location_Altitude: 335
missing values in Altitude: 0
missing values in Q1__Location_Accuracy: 335
missing values in Accuracy: 0


Removing Location_ columns from senegal data


In [812]:
senegal_df = senegal_df.drop(['Q1__Location_Latitude__continuous', 'Q1__Location_Longitude__continuous', 'Q1__Location_Altitude__continuous', 'Q1__Location_Accuracy__continuous'], axis=1)

Both are ok for nepal

In [813]:
for column in ['Latitude', 'Longitude', 'Altitude', 'Accuracy']:
    print(f"missing values in Location_{column}: {nepal_df[f'Q1__Location_{column}__continuous'].isna().sum()}")
    print(f"missing values in {column}: {nepal_df[f'Q1__{column}__continuous'].isna().sum()}")

missing values in Location_Latitude: 0
missing values in Latitude: 0
missing values in Location_Longitude: 0
missing values in Longitude: 0
missing values in Location_Altitude: 0
missing values in Altitude: 0
missing values in Location_Accuracy: 0
missing values in Accuracy: 0


Still removing from nepal Location_ columns for consistency between both datasets

In [814]:
nepal_df = nepal_df.drop(['Q1__Location_Latitude__continuous', 'Q1__Location_Longitude__continuous', 'Q1__Location_Altitude__continuous', 'Q1__Location_Accuracy__continuous'], axis=1)

Defining function to analyze locational columns

In [815]:

from folium.plugins import MarkerCluster

def lat_lon_hillshade_map(df, zoom_start=11):
    """
    Folium map with:
      • Default OpenStreetMap layer
      • ESRI World Hillshade (true terrain) layer
      • MarkerCluster of your points with Altitude pop-ups
      • LayerControl for toggling back and forth
    """
    # 1) Prepare & center
    df_map = df.dropna(subset=['Q1__Latitude__continuous','Q1__Longitude__continuous']).copy()
    center = [df_map['Q1__Latitude__continuous'].mean(), df_map['Q1__Longitude__continuous'].mean()]

    # 2) Build base map (OSM by default)
    m = folium.Map(location=center, zoom_start=zoom_start)

    # 3) Add ESRI World Hillshade as a second base layer
    folium.TileLayer(
        tiles='https://server.arcgisonline.com/ArcGIS/rest/services/'
              'Elevation/World_Hillshade/MapServer/tile/{z}/{y}/{x}',
        attr='Tiles © Esri — Source: USGS',
        name='Hillshade (ESRI)'
    ).add_to(m)

    # 4) Your markers (clustered)
    mc = MarkerCluster(name='Points').add_to(m)
    for idx, row in df_map.iterrows():
        folium.Marker(
            location=[row['Q1__Latitude__continuous'], row['Q1__Longitude__continuous']],
            popup=(
                f"<b>Index:</b> {idx}<br>"
                f"<b>Altitude:</b> {row.get('Q1__Altitude__continuous','N/A')} m"
            )
        ).add_to(mc)

    # 5) Toggle control
    folium.LayerControl(collapsed=False).add_to(m)

    return m


Analyze lat long data for nepal, both openstreetmap and hillshade for elevation

In [816]:
if PLOT_GRAPHS:
    nepal_map = lat_lon_hillshade_map(nepal_df)
    nepal_map   # in Jupyter this will render with two base‐layers to switch between


The same for Senegal

In [817]:
if PLOT_GRAPHS:
    print("SENEGAL DATA")
    senegal_map = lat_lon_hillshade_map(senegal_df)
    senegal_map


Check against google if the altitude column is accurate given the lat lot coordinates, plot accuracy also to see if there is some connection

In [818]:
import requests

def plot_altitude_error(df, lat_col='Q1__Latitude__continuous', lon_col='Q1__Longitude__continuous', alt_col='Q1__Altitude__continuous', accuracy_col='Q1__Accuracy__continuous', api_url='https://api.open-elevation.com/api/v1/lookup', batch_size=100, bins=30):
    '''
    Fetches true elevations and plots a combined view showing:
      - Histogram of altitude errors (recorded - true).
      - Median reported accuracy for each error bin overlaid as a line.

    Parameters:
    df (pd.DataFrame): DataFrame with latitude, longitude, recorded altitude, and accuracy columns.
    lat_col (str): Name of the latitude column.
    lon_col (str): Name of the longitude column.
    alt_col (str): Name of the recorded altitude column.
    accuracy_col (str): Name of the accuracy column.
    api_url (str): Elevation API endpoint.
    batch_size (int): Number of locations per API request.
    bins (int): Number of bins for error histogram and averaging.

    Returns:
    None. Displays a joint plot of error distribution and median accuracy per error bin.
    '''
    # Work on a copy
    df = df.copy()
    true_alts = []

    # Batch request loop to fetch true elevations
    for i in range(0, len(df), batch_size):
        batch = df.iloc[i:i+batch_size]
        locations = '|'.join(f"{lat},{lon}" for lat, lon in zip(batch[lat_col], batch[lon_col]))
        resp = requests.get(api_url, params={'locations': locations})
        resp.raise_for_status()
        results = resp.json().get('results', [])
        if len(results) != len(batch):
            raise ValueError(f"Expected {len(batch)} results, got {len(results)}")
        true_alts.extend(r['elevation'] for r in results)

    # Compute errors and extract accuracy
    recorded = df[alt_col].values
    errors = recorded - np.array(true_alts)
    accuracies = df[accuracy_col].values

    # Prepare bins and compute median accuracy per bin
    bin_edges = np.linspace(errors.min(), errors.max(), bins + 1)
    bin_centers = 0.5 * (bin_edges[:-1] + bin_edges[1:])
    digitized = np.digitize(errors, bin_edges) - 1
    median_acc = [np.median(accuracies[digitized == i]) if np.any(digitized == i) else np.nan for i in range(bins)]

    # Create combined plot
    fig, ax1 = plt.subplots()

    # Histogram of errors
    ax1.hist(errors, bins=bin_edges, alpha=0.6)
    ax1.set_xlabel('Altitude Error (m)')
    ax1.set_ylabel('Frequency')
    ax1.axvline(0, color='k', linestyle='--')


    plt.title('Altitude Error Distribution with Median Accuracy per Error Bin')
    plt.show()

In [819]:
if PLOT_GRAPHS:
    plot_altitude_error(nepal_df)

In [820]:
if PLOT_GRAPHS:
    plot_altitude_error(senegal_df)

Looks like gaussian noise error, just keep these columns

## Add resilience index

In [821]:
def add_abs_features(
    df: pd.DataFrame,
    lat_col: str = "lat",
    lon_col: str = "lon",
    water_thresh: float = 100,   # meters
    toilet_thresh: float = 50,    # meters
    elec_thresh: float = 100      # meters
) -> pd.DataFrame:
    """
    Enrich any DataFrame with (lat,lon) cols by adding:
      • abs_water_access  (0/1)
      • abs_toilet        (0/1)
      • abs_electricity   (0/1)
    plus these distance columns (in meters):
      dist_water_source, dist_toilet, dist_power_line,
      dist_primary_school, dist_health_facility,
      dist_livestock_market, dist_agri_market,
      dist_public_transport
    """
    # 1. Rename & build GeoDataFrame in EPSG:4326 → project to EPSG:3857 for meters
    df2 = df.rename(columns={lat_col: "lat", lon_col: "lon"}).copy()
    gdf = gpd.GeoDataFrame(
        df2,
        geometry=gpd.points_from_xy(df2.lon, df2.lat),
        crs="EPSG:4326"
    ).to_crs(epsg=3857)

    def add_osm_distance(gdf, tags: dict, col: str):
        # compute lat/lon bbox from projected gdf
        north, south, east, west = gdf.to_crs(epsg=4326).total_bounds[[3,1,2,0]]
        pad = 0.02  # degrees
        north += pad; south -= pad; east += pad; west -= pad
        bbox = (west, south, east, north)  # (left, bottom, right, top)

        # fetch POIs within bbox; signature is features_from_bbox(bbox, tags) :contentReference[oaicite:1]{index=1}
        pois = ox.features_from_bbox(bbox, tags)

        if pois.empty:
            gdf[col] = np.nan
        else:
            # merge all geometries to speed up distance queries
            geom_union = pois.to_crs(epsg=3857).geometry.unary_union
            gdf[col] = gdf.geometry.apply(lambda p: p.distance(geom_union))
        return gdf

    # 2. Compute each service distance
    gdf = add_osm_distance(gdf, {"amenity":"drinking_water"},     "dist_water_source")
    gdf = add_osm_distance(gdf, {"amenity":"toilets"},            "dist_toilet")
    gdf = add_osm_distance(gdf, {"power":"line"},                 "dist_power_line")
    gdf = add_osm_distance(gdf, {"amenity":"school"},             "dist_primary_school")
    gdf = add_osm_distance(gdf, {"amenity":["hospital","clinic"]},"dist_health_facility")
    gdf = add_osm_distance(gdf, {"amenity":"marketplace"},        "dist_livestock_market")
    gdf = add_osm_distance(gdf, {"shop":"greengrocer"},           "dist_agri_market")
    # public transport = nearest of bus_station or bus_stop
    gdf = add_osm_distance(gdf, {"amenity":"bus_station"},        "dist_bus_station")
    gdf = add_osm_distance(gdf, {"highway":"bus_stop"},           "dist_bus_stop")
    gdf["dist_public_transport"] = gdf[["dist_bus_station","dist_bus_stop"]].min(axis=1)

    # 4. Return original + ABS columns
    cols = [
      "dist_water_source","dist_toilet","dist_power_line",
      "dist_primary_school","dist_health_facility",
      "dist_livestock_market","dist_agri_market","dist_public_transport"
    ]
    df2 = df.rename(columns={"lat": lat_col, "lon": lon_col}).copy()
    return pd.concat([df2.reset_index(drop=True), gdf[cols].reset_index(drop=True)], axis=1)

In [822]:
nepal_df = add_abs_features(
    nepal_df,
    lat_col="Q1__Latitude__continuous",
    lon_col="Q1__Longitude__continuous"
)

senegal_df = add_abs_features(
    senegal_df,
    lat_col="Q1__Latitude__continuous",
    lon_col="Q1__Longitude__continuous"
)

C:\Users\nadav\AppData\Local\Temp\ipykernel_30844\2725292146.py:42: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  geom_union = pois.to_crs(epsg=3857).geometry.unary_union
C:\Users\nadav\AppData\Local\Temp\ipykernel_30844\2725292146.py:42: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  geom_union = pois.to_crs(epsg=3857).geometry.unary_union
C:\Users\nadav\AppData\Local\Temp\ipykernel_30844\2725292146.py:42: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  geom_union = pois.to_crs(epsg=3857).geometry.unary_union
C:\Users\nadav\AppData\Local\Temp\ipykernel_30844\2725292146.py:42: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  geom_union = pois.to_crs(epsg=3857).geometry.unary_union
C:\Users\nadav\AppData\Local\Temp\ipykernel_30844\2725292146.py:42: DeprecationWarning: 

In [823]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

def compute_sustainable_livelihood_score(
    df: pd.DataFrame,
    country: str
) -> pd.DataFrame:
    """
    Compute a 0–1 'sustainable_livelihood_score' as a formative composite
    across five livelihood domains, using PCA(1+2) per domain where needed.
    """
    df = df.copy()
    country = country.lower()
    
    # ── 1. Domain → feature lists ──────────────────────────────────────────
    if country == "nepal":
        domain_features = {
            "natural": [
                "Q50__How_much_land_that_is_yours_do_you_cultivate_bigha__continuous",
                "Q51__How_much_land_that_is_rented_or_leased_do_you_cultivate_bigha__continuous",
                "Q52__On_how_much_land_do_you_grow_vegetables_bigha__continuous",
                "Q62__How_much_VEGETABLES_do_you_harvest_per_year_from_this_plot_kilograms__continuous",
                "Q0__TOTAL_AREA__continuous"
            ],
            "physical": [
                "Q55__For_Vegetables_do_you_use_improved_and_or_variety_seeds_yes__binary__1",
                "Q57__If_you_self_prepare_seedlings_how_Nursery__binary__1",
                "Q60__do_you_use_pesticides_or_herbicides_on_this_plot_Yes__binary__1",
                "Q78__How_many_years_have_you_been_using_drip_irrigation_if_no_zero__continuous",
                "dist_water_source","dist_toilet","dist_power_line",
                "dist_primary_school","dist_health_facility",
                "dist_livestock_market","dist_agri_market","dist_public_transport"
            ],
            "financial": [
                "Q108__What_is_your_households_yearly_income_from_agriculture_NPR__continuous",
                "Q109__What_is_your_households_yearly_income_overall_including_agriculture_NPR__continuous",
                "Q110__Do_you_currently_have_a_loan_if_no_zero_if_yes_amount_of_money_loaned_NPR__continuous",
                "Q70__in_the_past_12_months_did_you_receive_any_info_from_anyone_on_agriculture__binary__1"
            ],
            "human": [
                "Q101__how_many_people_live_in_this_household__continuous",
                "Q5__AgeYears__continuous",
                "Q106__Education_level_of_this_person_that_is_interviewed_years_of_formal_education__continuous",
                "Q107__Education_level_of_your_wife_husband_How_many_years_of_formal_education__continuous"
            ],
            "social": [
                "Q0__hope_total__continuous",
                "Q0__self_control_score__continuous"
            ]
        }

    elif country == "senegal":
        domain_features = {
            "natural": [
                "Q60__Land_owned_cultivated_ha__continuous",
                "Q61__Land_rented_cultivated_ha__continuous",
                "Q62__Land_grow_vegetables_ha__continuous",
                "Q71__VEG_harvest_per_year_kg__continuous"
            ],
            "physical": [
                "Q69__Use_pesticide_or_herbicide__binary__1",
                "Q84__Years_using_drip_irrigation__continuous",
                "Q76__Received_agri_info_last_12m__binary__1",
                "dist_water_source","dist_toilet","dist_power_line",
                "dist_primary_school","dist_health_facility",
                "dist_livestock_market","dist_agri_market","dist_public_transport"
            ],
            "financial": [
                "Q90__Yearly_income_agriculture_XOF__continuous",
                "Q91__Yearly_income_overall_XOF__continuous",
                "Q92__Current_loan_amount_XOF__continuous"
            ],
            "human": [
                "Q87__Household_size__continuous",
                "Q5__Age__continuous",
                "Q0__Education_years__continuous"
            ],
            "social": [
                "Q0__Hope_total__continuous",
                "Q93__Trust_most_people__ordinal"
            ]
        }
    else:
        raise ValueError("country must be 'nepal' or 'senegal'")

    # ── 2. Summarize each domain with PCA(1) or PCA(1+2) ────────────────────
    domain_evr    = {}
    domain_scores = pd.DataFrame(index=df.index)

    for domain, feats in domain_features.items():
        # a) subset, fill, standardize
        X  = df[feats].copy().fillna(df[feats].mean())
        Xs = StandardScaler().fit_transform(X)

        # b) fit two PCs
        pca      = PCA(n_components=2, random_state=0).fit(Xs)
        evr1,evr2 = pca.explained_variance_ratio_
        comp1,comp2 = pca.transform(Xs).T

        # c) if PC1 <50% capture, fold in PC2
        if evr1 < 0.50:
            w1 = evr1 / (evr1 + evr2)
            w2 = evr2 / (evr1 + evr2)
            score = w1 * comp1 + w2 * comp2
            domain_evr[domain] = evr1 + evr2
        else:
            score = comp1
            domain_evr[domain] = evr1

        domain_scores[f"{domain}_score"] = score

    # ── 3. Composite: variance‐weighted sum of domain scores ────────────────
    total_evr = sum(domain_evr.values())
    weights   = {d: evr / total_evr for d, evr in domain_evr.items()}

    # build composite score
    composite = sum(domain_scores[f"{d}_score"] * w for d,w in weights.items())
    df["Q0__sustainable_livelihood_score__continuous"] = composite

    return df


# ── Usage ───────────────────────────────────────────────────────────────
nepal_df   = compute_sustainable_livelihood_score(nepal_df,   country="nepal")
senegal_df = compute_sustainable_livelihood_score(senegal_df, country="senegal")

print(nepal_df[["Q0__sustainable_livelihood_score__continuous"]].head())
print(senegal_df[["Q0__sustainable_livelihood_score__continuous"]].head())

   Q0__sustainable_livelihood_score__continuous
0                                      0.757433
1                                      0.642556
2                                      1.087853
3                                      0.801880
4                                     -0.261796
   Q0__sustainable_livelihood_score__continuous
0                                      0.420186
1                                      0.680015
2                                      0.011791
3                                      0.278026
4                                     -0.860718


## Analyze All Other Features

Define a function to plot the marginal distributions of the remaining features, bar plots used for categorical features and histograms for numeric features

In [824]:
from collections import defaultdict

def plot_marginals(df: pd.DataFrame):
    # Group columns by question
    groups = defaultdict(list)
    for col in df.columns:
        meta = parse_feature_metadata(col)
        if meta:
            groups[meta["qid"]].append(col)

    for qid, cols in groups.items():
        meta0 = parse_feature_metadata(cols[0])
        ftype = meta0["type"]
        feat = meta0["name"].replace('_', ' ')

        plt.figure()
        if ftype == "binary":
            # ---- single‐select (<=2 columns) ----
            if len(cols) <= 2:
                # pick the “yes” column: if only one, use that; if two, look for '-1'
                if len(cols) == 1:
                    yes_col = cols[0]
                else:
                    yes_col = next(c for c in cols if c.endswith('-1'))
                ser = df[yes_col].fillna(0).astype(int)
                ser.value_counts().sort_index().plot(kind="bar", rot=0)
                plt.title(f"{qid} {feat} (binary yes/no)")
                plt.xlabel(feat)
                plt.ylabel("Count")

            # ---- multi‐select (>2 columns) ----
            else:
                counts = df[cols].fillna(0).astype(int).sum(axis=0)
                labels = [parse_feature_metadata(c)["name"].replace('_',' ')
                          for c in cols]
                pd.Series(counts.values, index=labels)\
                  .sort_values(ascending=False)\
                  .plot(kind="bar", rot=45)
                plt.title(f"{qid} {feat} (multi-select)")
                plt.xlabel(feat)
                plt.ylabel("Count")

        else:
            # continuous/ordinal/nominal
            ser = df[cols[0]]
            if pd.api.types.is_numeric_dtype(ser):
                plt.hist(ser.dropna(), bins='auto')
                plt.title(f"{qid} {feat} ({ftype})\n"
                          f"mean={ser.mean():.2f}, std={ser.std():.2f}")
            else:
                ser.value_counts().sort_index().plot(kind="bar", rot=0)
                plt.title(f"{qid} {feat} ({ftype})")
                plt.xlabel(feat)
                plt.ylabel("Count")

        plt.tight_layout()
        plt.show()

In [825]:
if PLOT_GRAPHS:
    plot_marginals(nepal_df)

In [826]:
if PLOT_GRAPHS:
    plot_marginals(senegal_df)

# Continuous & Discrete Missing Values

In [827]:
# dropping non Q#... columns
nepal_df = drop_non_relevant_columns(nepal_df)
senegal_df = drop_non_relevant_columns(senegal_df)

Get all continuous columns

In [828]:
non_cat_cols_nepal = []
non_cat_cols_senegal = []
for col in nepal_df.columns:
    meta_type = parse_feature_metadata(col)['type']
    if meta_type == "continuous":
        non_cat_cols_nepal.append(col)

for col in senegal_df.columns:

    meta_type = parse_feature_metadata(col)['type']
    
    if meta_type == "continuous":
        non_cat_cols_senegal.append(col)
    

Check how many missing values per column

In [829]:
missing_counts_nepal = nepal_df[non_cat_cols_nepal].isnull().sum()
print("Missing values per non-categorical column in nepal:")
print(missing_counts_nepal)

missing_counts_senegal = senegal_df[non_cat_cols_senegal].isnull().sum()
print("Missing values per non-categorical column in senegal:")
print(missing_counts_senegal)

Missing values per non-categorical column in nepal:
Q1__Latitude__continuous                                                                           0
Q1__Longitude__continuous                                                                          0
Q1__Altitude__continuous                                                                           0
Q1__Accuracy__continuous                                                                           0
Q5__AgeYears__continuous                                                                           0
Q50__How_much_land_that_is_yours_do_you_cultivate_bigha__continuous                                0
Q51__How_much_land_that_is_rented_or_leased_do_you_cultivate_bigha__continuous                     0
Q52__On_how_much_land_do_you_grow_vegetables_bigha__continuous                                     1
Q62__How_much_VEGETABLES_do_you_harvest_per_year_from_this_plot_kilograms__continuous              0
Q78__How_many_years_have_you_been_using

Check if missingness of How_many_children_do_you... columns in nepal are random

In [830]:
children_cols = ['Q102__How_many_children_do_you_have__continuous', 'Q103__How_many_children_do_you_plan_to_have_in_your_whole_live__continuous']
y_col = 'Q0__target2_yearly_income_from_agr_USD__continuous'

if PLOT_GRAPHS:
    for col in children_cols:

        mask_missing = nepal_df[col].isnull().values

        # Choose colors for each point
        colors = np.where(mask_missing, 'red', 'blue')

        plt.figure(figsize=(8,4))
        plt.scatter(nepal_df.index, nepal_df[y_col], c=colors, alpha=0.7)
        plt.xlabel("Index")
        plt.ylabel("Yearly Income from Agriculture (USD)")
        plt.title("Yearly Income from Agriculture (USD) vs. index\n\nRed = 'how many children' feature was missing")
        plt.show()

Missingness is uniformly spread throughout. We remove the columns because our coordinator
said we should

In [831]:
nepal_df = nepal_df.drop(columns=children_cols)


In [832]:

for col in children_cols:
    non_cat_cols_nepal.remove(col)

For the rest, fill missing values with median since very small amount of them and don't want to lose farmers

In [833]:
nepal_df[non_cat_cols_nepal] = nepal_df[non_cat_cols_nepal].fillna(nepal_df[non_cat_cols_nepal].median())
senegal_df[non_cat_cols_senegal] = senegal_df[non_cat_cols_senegal].fillna(senegal_df[non_cat_cols_senegal].median())

Recheck missing values in both datasets

In [834]:
# Check how many missing values per column
missing_counts_nepal = nepal_df[non_cat_cols_nepal].isnull().sum()
print("Missing values per non-categorical column in nepal:")
print(missing_counts_nepal)

missing_counts_senegal = senegal_df[non_cat_cols_senegal].isnull().sum()
print("Missing values per non-categorical column in senegal:")
print(missing_counts_senegal)

Missing values per non-categorical column in nepal:
Q1__Latitude__continuous                                                                          0
Q1__Longitude__continuous                                                                         0
Q1__Altitude__continuous                                                                          0
Q1__Accuracy__continuous                                                                          0
Q5__AgeYears__continuous                                                                          0
Q50__How_much_land_that_is_yours_do_you_cultivate_bigha__continuous                               0
Q51__How_much_land_that_is_rented_or_leased_do_you_cultivate_bigha__continuous                    0
Q52__On_how_much_land_do_you_grow_vegetables_bigha__continuous                                    0
Q62__How_much_VEGETABLES_do_you_harvest_per_year_from_this_plot_kilograms__continuous             0
Q78__How_many_years_have_you_been_using_drip_irr

# Outliers

Check skew and kurtosis of features

In [835]:
def print_per_col_skew_and_kurtosis(df, cols):
    desc = pd.DataFrame(index=cols, columns=['skewness','kurtosis'])
    desc['skewness'] = df[cols].skew()
    desc['kurtosis'] = df[cols].kurtosis()
    print(desc)

print("For nepal dataset:")
print_per_col_skew_and_kurtosis(nepal_df, non_cat_cols_nepal)




For nepal dataset:
                                                     skewness    kurtosis
Q1__Latitude__continuous                             2.501469   11.458427
Q1__Longitude__continuous                           -2.078695    3.327960
Q1__Altitude__continuous                             7.738261   61.181298
Q1__Accuracy__continuous                             1.245955    1.722112
Q5__AgeYears__continuous                             0.011752   -0.698653
Q50__How_much_land_that_is_yours_do_you_cultiva...   0.380098   -0.503872
Q51__How_much_land_that_is_rented_or_leased_do_...   2.428187    7.002975
Q52__On_how_much_land_do_you_grow_vegetables_bi...   1.379596    1.564302
Q62__How_much_VEGETABLES_do_you_harvest_per_yea...   7.498011   80.246841
Q78__How_many_years_have_you_been_using_drip_ir...  13.138456  180.751947
Q101__how_many_people_live_in_this_household__c...   1.779578    7.940154
Q106__Education_level_of_this_person_that_is_in...  -0.183514    0.424381
Q107__Education_lev

In [836]:
print("For senegal dataset:")
print_per_col_skew_and_kurtosis(senegal_df, non_cat_cols_senegal)

For senegal dataset:
                                                     skewness    kurtosis
Q1__Latitude__continuous                             0.651947   -1.027130
Q1__Longitude__continuous                            0.434510   -1.374106
Q1__Altitude__continuous                            -0.027704    1.898947
Q1__Accuracy__continuous                             2.523551    6.847364
Q5__Age__continuous                                  0.067253   -0.843279
Q0__Hope_total__continuous                           0.911953    1.005143
Q0__Average_CS__continuous                           0.943979    0.344805
Q60__Land_owned_cultivated_ha__continuous            0.261807   -1.177306
Q61__Land_rented_cultivated_ha__continuous           4.403219   20.345663
Q62__Land_grow_vegetables_ha__continuous             0.228387   -1.165957
Q71__VEG_harvest_per_year_kg__continuous             2.712008   10.480210
Q84__Years_using_drip_irrigation__continuous         2.467766    5.748366
Q87__Household_si

A lot of features have very long and heavy tails, would like also a visual understanding, we plot qq plots aswell for the non-categorical columns

In [837]:
# Get QQ plots

import scipy.stats as stats

print("nepal qq plots")
if PLOT_GRAPHS:
    for col in non_cat_cols_nepal:
        plt.figure()
        stats.probplot(nepal_df[col].dropna(), dist="norm", plot=plt)
        plt.title(f"Q–Q plot for {col}")
        plt.show()

nepal qq plots


In [838]:
print("senegal qq plots")
if PLOT_GRAPHS:
    for col in non_cat_cols_senegal:
        plt.figure()
        stats.probplot(senegal_df[col].dropna(), dist="norm", plot=plt)
        plt.title(f"Q–Q plot for {col}")
        plt.show()

senegal qq plots


Very assymetric and kurtosis high, would like to classify outliers in a robust manner, taking into account the shape of the empirical distribution, we will use robust z scores calculated using the Qn scale

In [839]:
from statsmodels import robust

threshold = 3

def robust_z_scores(df, cols):

    outlier_counts = {}
    outlier_indices = {}
    robust_z_scores = {}

    for col in cols:
        # 1. Grab the non-null values as a 1D NumPy array
        data = df[col].dropna().values
        
        # 2. Compute Qn scale
        qn = robust.scale.qn_scale(data)

        # 3. Compute the column’s median
        med = np.median(data)

        # 4. Compute “robust z‐scores” relative to Qn
        #    (Note: unlike classical z, this is robust to heavy tails & skew)
        robust_z = (data - med) / qn
        robust_z_scores[col] = robust_z
        
        # 5. Flag outliers
        mask = np.abs(robust_z) > threshold
        
        # 6. Store counts and (optionally) original DataFrame indices
        outlier_counts[col] = int(mask.sum())
        # to get the original df indices:
        outlier_indices[col] = df[col].dropna().index[mask].tolist()

    # Convert to a nice summary DataFrame
    summary = pd.DataFrame.from_dict({
        'n_outliers': outlier_counts,
        'pct_outliers': {c: outlier_counts[c] / df[c].count() * 100 
                        for c in cols}
    }).T

    print("Outliers per column (using Qn):")
    print(summary)

    return outlier_indices, robust_z_scores

In [840]:
print("applied robust z-score outlier flagging to nepal:")
outlier_indices_nepal, robust_z_scores_nepal = robust_z_scores(nepal_df, non_cat_cols_nepal)

applied robust z-score outlier flagging to nepal:
Outliers per column (using Qn):
              Q1__Latitude__continuous  Q1__Longitude__continuous  \
n_outliers                   43.000000                  48.000000   
pct_outliers                 16.044776                  17.910448   

              Q1__Altitude__continuous  Q1__Accuracy__continuous  \
n_outliers                    9.000000                 11.000000   
pct_outliers                  3.358209                  4.104478   

              Q5__AgeYears__continuous  \
n_outliers                         0.0   
pct_outliers                       0.0   

              Q50__How_much_land_that_is_yours_do_you_cultivate_bigha__continuous  \
n_outliers                                           155.000000                     
pct_outliers                                          57.835821                     

              Q51__How_much_land_that_is_rented_or_leased_do_you_cultivate_bigha__continuous  \
n_outliers                

C:\Users\nadav\AppData\Local\Temp\ipykernel_30844\2071753757.py:23: RuntimeWarning: divide by zero encountered in divide
  robust_z = (data - med) / qn
C:\Users\nadav\AppData\Local\Temp\ipykernel_30844\2071753757.py:23: RuntimeWarning: invalid value encountered in divide
  robust_z = (data - med) / qn


In [841]:
print("applied robust z-score outlier flagging to senegal:")
outlier_indices_senegal, robust_z_scores_senegal = robust_z_scores(senegal_df, non_cat_cols_senegal)

applied robust z-score outlier flagging to senegal:
Outliers per column (using Qn):
              Q1__Latitude__continuous  Q1__Longitude__continuous  \
n_outliers                   72.000000                 106.000000   
pct_outliers                 21.492537                  31.641791   

              Q1__Altitude__continuous  Q1__Accuracy__continuous  \
n_outliers                    5.000000                 28.000000   
pct_outliers                  1.492537                  8.358209   

              Q5__Age__continuous  Q0__Hope_total__continuous  \
n_outliers                    0.0                   20.000000   
pct_outliers                  0.0                    5.970149   

              Q0__Average_CS__continuous  \
n_outliers                     15.000000   
pct_outliers                    4.477612   

              Q60__Land_owned_cultivated_ha__continuous  \
n_outliers                                          0.0   
pct_outliers                                        0.0 

C:\Users\nadav\AppData\Local\Temp\ipykernel_30844\2071753757.py:23: RuntimeWarning: divide by zero encountered in divide
  robust_z = (data - med) / qn
C:\Users\nadav\AppData\Local\Temp\ipykernel_30844\2071753757.py:23: RuntimeWarning: invalid value encountered in divide
  robust_z = (data - med) / qn


Crosschecking with the QQ plots and marginal distributions of a lot of these features, many of these values classified as outliers make sense in terms of the data and should not be classified as such (especially in the ordinal columns), took a handful of columns frome each dataset for which there could be outliers, calculated robust z scores in order to identify outliers (greater than threshold=3.0). For the productivity column outliers we simply remove rows with outliers. For any of the other columns we perform winsorization, defining the caps using the qn scale used to calculate the robust z scores (med +- threshold*qn)

In [842]:
outlier_cols_nepal = ['Q62__How_much_VEGETABLES_do_you_harvest_per_year_from_this_plot_kilograms__continuous',
                      'Q78__How_many_years_have_you_been_using_drip_irrigation_if_no_zero__continuous',
                      'Q108__What_is_your_households_yearly_income_from_agriculture_NPR__continuous',
                      'Q109__What_is_your_households_yearly_income_overall_including_agriculture_NPR__continuous',
                      'Q101__how_many_people_live_in_this_household__continuous',
                      'Q0__self_control_score__continuous', 
                      'Q0__hope_total__continuous',
                      'Q0__positive_total__continuous',
                      'Q0__negative_total__continuous',
                      'Q0__Positive_Negative_Score__continuous',]

outlier_cols_senegal = ['Q0__Hope_total__continuous',
                        'Q71__VEG_harvest_per_year_kg__continuous',
                        'Q87__Household_size__continuous',
                        'Q90__Yearly_income_agriculture_XOF__continuous',
                        'Q91__Yearly_income_overall_XOF__continuous',
                        'Q92__Current_loan_amount_XOF__continuous',]


target_cols = ['Q0__target1_yearly_income_from_agr_per_land_SQM__continuous',
               'Q0__target2_yearly_income_from_agr_USD__continuous',
               'Q0__target3_veg_per_area__continuous']

def handle_outliers(
    df: pd.DataFrame,
    outlier_cols: list[str],
    target_cols:  list[str],
    threshold:   float = 3.0
) -> pd.DataFrame:
    """
    1. Remove rows where any column in `target_cols` has |robust z-score| > threshold.
    2. For each column in `outlier_cols`, winsorize at median ± threshold * Qn scale.

    Parameters
    ----------
    df : DataFrame
      Original data.
    outlier_cols : list of str
      Columns to winsorize.
    target_cols : list of str
      Columns whose outliers trigger row removal.
    threshold : float
      Robust z-score cutoff (uses Qn scale from statsmodels.robust.scale).
    """
    df2 = df.copy()

    # ── Step 1: Drop rows with extreme targets ──────────────────────────────
    drop_mask = pd.Series(False, index=df2.index)
    for col in target_cols:
        vals = df2[col].dropna().values
        if len(vals) == 0:
            continue

        med = np.median(vals)
        qn  = robust.scale.qn_scale(vals)
        if qn == 0:
            continue

        z = (df2[col] - med) / qn
        drop_mask |= z.abs() > threshold

    df2 = df2.loc[~drop_mask].reset_index(drop=True)

    # ── Step 2: Winsorize outlier_cols ─────────────────────────────────────
    for col in outlier_cols:
        series = df2[col]
        non_na = series.dropna()
        if non_na.empty:
            continue

        med = np.median(non_na.values)
        qn  = robust.scale.qn_scale(non_na.values)
        if qn == 0:
            continue

        # define the cap bounds
        lower_cap = med - threshold * qn
        upper_cap = med + threshold * qn

        # winsorize: anything below lower_cap → lower_cap; above upper_cap → upper_cap
        df2[col] = series.clip(lower=lower_cap, upper=upper_cap)

    return df2

In [843]:
nepal_df = handle_outliers(nepal_df, outlier_cols_nepal, target_cols)
senegal_df = handle_outliers(senegal_df, outlier_cols_senegal, target_cols)




Get all categorical and binary columns

In [844]:
cat_cols_nepal = []
cat_cols_senegal = []
for col in nepal_df.columns:
    meta_type = parse_feature_metadata(col)['type']
    if meta_type not in ['continuous', 'discrete', 'time']:
        cat_cols_nepal.append(col)

for col in senegal_df.columns:
    meta_type = parse_feature_metadata(col)['type']
    if meta_type not in ['continuous', 'discrete', 'time']:
        cat_cols_senegal.append(col)

Check that none have an overly large amount of categories

In [845]:
unique_counts = nepal_df[cat_cols_nepal].nunique()
# 3. see which columns are OK (≤20) or not (>20)
ok      = unique_counts[ unique_counts <= 20 ]
too_many = unique_counts[ unique_counts  > 20 ]

print("Columns with ≤20 categories:\n", ok.sort_values(ascending=False))
print("\nColumns with >20 categories:\n", too_many.sort_values(ascending=False))


Columns with ≤20 categories:
 Q105__main_sources_of_income__nominal                                                          19
Q53__What_type_of_crop_is_grown_on_this_plot__nominal                                          14
Q65__Do_you_do_any_of_the_following__nominal                                                   12
Q8__WardNumber__nominal                                                                        10
Q21__Hope_5_I_am_easily_downed_being_at_a_low_position_brought_down_in_an_argument__ordinal     9
                                                                                               ..
Q55__For_Vegetables_do_you_use_improved_and_or_variety_seeds_yes__binary__1                     2
Q57__If_you_self_prepare_seedlings_how_Nursery__binary__1                                       2
Q70__in_the_past_12_months_did_you_receive_any_info_from_anyone_on_agriculture__binary__1       2
Q58__fertilizer_on_this_plot__nominal                                                   

In [846]:
unique_counts = senegal_df[cat_cols_senegal].nunique()
# 3. see which columns are OK (≤20) or not (>20)
ok      = unique_counts[ unique_counts <= 20 ]
too_many = unique_counts[ unique_counts  > 20 ]

print("Columns with ≤20 categories:\n", ok.sort_values(ascending=False))
print("\nColumns with >20 categories:\n", too_many.sort_values(ascending=False))


Columns with ≤20 categories:
 Q7__name_of_village__nominal                                         16
Q14__Hope_4_There_are_lots_of_ways_around_any_problem__ordinal        9
Q13__Hope_3_I_feel_tired_most_of_the_time__ordinal                    8
Q12__Hope_2_I_energetically_pursue_my_goals__ordinal                  8
Q11__Hope_1_I_can_think_of_many_ways_to_get_out_of_a_jam__ordinal     8
                                                                     ..
Q76__Received_agri_info_last_12m__binary__1                           2
Q0__Has_Education_Yes_No__binary__1                                   2
Q86__Sex_male__binary__1                                              2
Q69__Use_pesticide_or_herbicide__binary__1                            2
Q63__CROP__nominal                                                    2
Length: 73, dtype: int64

Columns with >20 categories:
 Q4__Phone_Number__nominal    92
Q73__Machinery__nominal      26
dtype: int64


Only telephone, village and machinery columns have large amounts of categories which makes sense

# Constant Columns

We remove all remaining columns that have only a single value as this adds no new information

In [847]:
nepal_df = nepal_df.loc[:, nepal_df.nunique(dropna=False) > 1]
senegal_df = senegal_df.loc[:, senegal_df.nunique(dropna=False) > 1]

# Bivariate Analysis

Define the feature groups that we will be interested in throughout this project

In [848]:
NEPAL_CAT_COL_NAMES = {'Q11': 'Marital_Status',
 'Q53': 'What_type_of_crop_is_grown_on_this_plot',
 'Q54': 'For_vegetables_what_is_your_source_of_seeds',
 'Q56': 'For_vegetables_do_you_use_seedlings',
 'Q58': 'fertilizer_on_this_plot',
 'Q63': 'What_is_the_main_use_of_produce_from_holding',
 'Q64': 'Do_you_use_machinery_or_and_equipment_on_the_plot',
 'Q65': 'Do_you_do_any_of_the_following',
 'Q67': 'What_do_you_use_soil_analysis_for',
 'Q68': 'How_do_you_conduct_soil_analysis',
 'Q69': 'What_is_correct_for_you',
 'Q71': 'in_the_past_12_months_from_who_did_you_receive_info_on_agriculture',
 'Q73': 'Did_you_receive_anything_from_these_organizations',
 'Q74': 'How_do_you_decide_to_plow',
 'Q75': 'How_do_you_decide_to_begin_sowing',
 'Q76': 'What_type_of_irrigation_do_you_use',
 'Q100': 'Caste',
 'Q105': 'main_sources_of_income',
 'Q111': 'Generally_speaking_would_you_say_that_most_people_can_be_trusted',
 'Q112': 'I_am_much_better_than_most_farmers_here',
 'Q113': 'dislike_not_knowing_what_is_going_to_happen'}

SENEGAL_CAT_COL_NAMES = {'Q63': 'CROP',
 'Q64': 'Seed_source',
 'Q65': 'Variety',
 'Q66': 'Seedlings',
 'Q67': 'Fertilizer',
 'Q72': 'Sold_VEG',
 'Q73': 'Machinery',
 'Q74': 'Practice',
 'Q77': 'Info_source',
 'Q79': 'Did_you_receive_anything_from_the_specified_organizations',
 'Q80': 'Plow_weather',
 'Q81': 'Sow',
 'Q82': 'Irrigation',
 'Q88': 'family_main_sources_income',
 'Q89': 'education_level'}

In [849]:
# 1) Geographic features
geo_features_nepal = [
    "Q1__Latitude__continuous",
    "Q1__Longitude__continuous",
    "Q7__District__nominal",
    "Q8__WardNumber__nominal",
    "Q9__NameOfVillage__nominal",
    "Q1__Altitude__continuous",
    "Q1__Accuracy__continuous",
]

geo_features_senegal = [
    "Q1__Latitude__continuous",
    "Q1__Longitude__continuous",
    "Q6__Arrondissement__nominal",
    "Q7__name_of_village__nominal",
    "Q1__Altitude__continuous",
    "Q1__Accuracy__continuous",
]

# 2) Feelings items (PANAS)
feelings_features_nepal = [
    "Q30__To_what_extent_you_have_felt_this_way_during_the_past_week_interested__ordinal",
    "Q31__To_what_extent_you_have_felt_this_way_during_the_past_week_Distressed_worried__ordinal",
    "Q32__To_what_extent_you_have_felt_this_way_during_the_past_week_Excited__ordinal",
    "Q33__To_what_extent_you_have_felt_this_way_during_the_past_week_Upset__ordinal",
    "Q34__To_what_extent_you_have_felt_this_way_during_the_past_week_Strong__ordinal",
    "Q35__To_what_extent_you_have_felt_this_way_during_the_past_week_Guilty__ordinal",
    "Q36__To_what_extent_you_have_felt_this_way_during_the_past_week_Scared__ordinal",
    "Q37__To_what_extent_you_have_felt_this_way_during_the_past_week_Hostile__ordinal",
    "Q38__To_what_extent_you_have_felt_this_way_during_the_past_week_Enthusiastic__ordinal",
    "Q39__To_what_extent_you_have_felt_this_way_during_the_past_week_Proud__ordinal",
    "Q40__To_what_extent_you_have_felt_this_way_during_the_past_week_Irritable__ordinal",
    "Q41__To_what_extent_you_have_felt_this_way_during_the_past_week_Alert__ordinal",
    "Q42__To_what_extent_you_have_felt_this_way_during_the_past_week_Ashamed__ordinal",
    "Q43__To_what_extent_you_have_felt_this_way_during_the_past_week_Inspired__ordinal",
    "Q44__To_what_extent_you_have_felt_this_way_during_the_past_week_Nervous__ordinal",
    "Q45__To_what_extent_you_have_felt_this_way_during_the_past_week_Determined__ordinal",
    "Q46__To_what_extent_you_have_felt_this_way_during_the_past_week_Attentive__ordinal",
    "Q47__To_what_extent_you_have_felt_this_way_during_the_past_week_Jittery__ordinal",
    "Q48__To_what_extent_you_have_felt_this_way_during_the_past_week_Active__ordinal",
    "Q49__To_what_extent_you_have_felt_this_way_during_the_past_week_Afraid__ordinal",
]

# 3) Character-strength items (Senegal)
character_features_senegal = [
    "Q24__Stand_up_in_face_of_opposition__ordinal",
    "Q25__Never_quit_task_before_done__ordinal",
    "Q26__Always_keep_promises__ordinal",
    "Q27__Always_finish_what_I_start__ordinal",
    "Q28__Leader_treats_everyone_equally__ordinal",
    "Q29__Always_busy_with_something_interesting__ordinal",
    "Q30__Strength_help_group_work_together__ordinal",
    "Q31__Must_stand_up_for_beliefs__ordinal",
    "Q32__Finish_things_despite_obstacles__ordinal",
    "Q33__Everyones_rights_equally_important__ordinal",
    "Q34__Excited_by_many_activities__ordinal",
    "Q35__Always_coming_up_with_new_ways__ordinal",
    "Q36__People_describe_me_as_wise__ordinal",
    "Q37__Promises_can_be_trusted__ordinal",
    "Q38__Give_everyone_a_chance__ordinal",
    "Q39__Effective_leader_treats_same__ordinal",
    "Q40__Extremely_grateful_person__ordinal",
    "Q41__Look_forward_to_each_new_day__ordinal",
    "Q42__Friends_say_I_have_many_ideas__ordinal",
    "Q43__Always_stand_up_for_beliefs__ordinal",
    "Q44__True_to_own_values__ordinal",
    "Q45__Think_through_consequences_before_acting__ordinal",
    "Q46__Have_lots_of_energy__ordinal",
    "Q47__Find_interest_in_any_situation__ordinal",
    "Q48__Thinking_things_through_is_part_of_me__ordinal",
    "Q49__Original_thinker__ordinal",
    "Q50__Mature_view_on_life__ordinal",
    "Q51__Feel_thankful_for_what_I_have__ordinal",
    "Q52__Always_weigh_pros_and_cons__ordinal",
    "Q53__Very_careful_person__ordinal",
    "Q54__Try_to_have_good_reasons_for_decisions__ordinal",
    "Q55__Always_make_careful_choices__ordinal",
    "Q56__Feel_profound_appreciation_daily__ordinal",
    "Q57__Awaken_with_excitement__ordinal",
    "Q58__Others_consider_me_wise__ordinal",
    "Q59__Worth_listening_to_everyones_opinion__ordinal",
]

# 4) Demographics
demo_features_nepal = [
    "Q10__SexMale__binary__1",
    "Q11__Marital_Status__nominal",
]

demo_features_senegal = [
    "Q86__Sex_male__binary__1",
]

# 5) Hope scale
hope_features_nepal = [
    "Q17__Hope_1_I_Can_Think_Of_Many_Ways_To_Get_Out_Of_A_Jam__ordinal",
    "Q18__Hope_2_I_energetically_pursue_my_goals__ordinal",
    "Q19__Hope_3_I_feel_tired_most_of_the_time__ordinal",
    "Q20__Hope_4_There_are_lots_of_ways_around_any_problem__ordinal",
    "Q21__Hope_5_I_am_easily_downed_being_at_a_low_position_brought_down_in_an_argument__ordinal",
    "Q22__Hope_6_I_can_think_of_many_ways_to_get_the_things_in_life_that_are_important_to_me__ordinal",
    "Q23__Hope_7_I_worry_about_my_health__ordinal",
    "Q24__Hope_8_Even_when_others_get_discouraged_I_know_I_can_find_a_way_to_solve_the_problem__ordinal",
    "Q25__Hope_9_My_past_experiences_have_prepared_me_well_for_my_future__ordinal",
    "Q26__Hope_10_I_have_been_pretty_successful_in_life__ordinal",
    "Q27__Hope_11_I_usually_find_myself_worrying_about_something__ordinal",
    "Q28__Hope_12_I_meet_the_goals_that_I_set_for_myself__ordinal",
]

hope_features_senegal = [
    "Q11__Hope_1_I_can_think_of_many_ways_to_get_out_of_a_jam__ordinal",
    "Q12__Hope_2_I_energetically_pursue_my_goals__ordinal",
    "Q13__Hope_3_I_feel_tired_most_of_the_time__ordinal",
    "Q14__Hope_4_There_are_lots_of_ways_around_any_problem__ordinal",
    "Q15__Hope_5_I_am_easily_downed_in_an_argument__ordinal",
    "Q16__Hope_6_I_can_think_of_many_ways_to_get_the_things_in_life_that_are_important_to_me__ordinal",
    "Q17__Hope_7_I_worry_about_my_health__ordinal",
    "Q18__Hope_8_Even_when_others_get_discouraged_I_know_I_can_find_a_way__ordinal",
    "Q19__Hope_9_My_past_experiences_have_prepared_me_well_for_my_future__ordinal",
    "Q20__Hope_10_Ive_been_pretty_successful_in_life__ordinal",
    "Q21__Hope_11_I_usually_find_myself_worrying_about_something__ordinal",
    "Q22__Hope_12_I_meet_the_goals_that_I_set_for_myself__ordinal",
]

# 6) Crop & fertilizer
crop_fertilizer_features_nepal = [
    "Q53__What_type_of_crop_is_grown_on_this_plot__nominal",
    "Q54__For_vegetables_what_is_your_source_of_seeds__nominal",
    "Q55__For_Vegetables_do_you_use_improved_and_or_variety_seeds_yes__binary__1",
    "Q56__For_vegetables_do_you_use_seedlings__nominal",
    "Q57__If_you_self_prepare_seedlings_how_Nursery__binary__1",
    "Q58__fertilizer_on_this_plot__nominal",
]

crop_fertilizer_features_senegal = [
    "Q63__CROP__nominal",
    "Q64__Seed_source__nominal",
    "Q65__Variety__nominal",
    "Q66__Seedlings__nominal",
    "Q67__Fertilizer__nominal",
]

# 7) Plot-level practices
plot_practice_features_nepal = [
    "Q63__What_is_the_main_use_of_produce_from_holding__nominal",
    "Q64__Do_you_use_machinery_or_and_equipment_on_the_plot__nominal",
    "Q65__Do_you_do_any_of_the_following__nominal",
]

plot_practice_features_senegal = [
    "Q69__Use_pesticide_or_herbicide__binary__1",
    "Q72__Sold_VEG__nominal",
    "Q73__Machinery__nominal",
    "Q74__Practice__nominal",
]

soil_features_senegal = [
    "Q76__Received_agri_info_last_12m__binary__1",
    "Q77__Info_source__nominal",
    "Q78__Names_of_organizations__nominal",
    "Q79__Did_you_receive_anything_from_the_specified_organizations__nominal",
]

# 9) Plowing, sowing & irrigation
plow_irrigation_features_nepal = [
    "Q74__How_do_you_decide_to_plow__nominal",
    "Q75__How_do_you_decide_to_begin_sowing__nominal",
    "Q76__What_type_of_irrigation_do_you_use__nominal",
]

plow_irrigation_features_senegal = [
    "Q80__Plow_weather__nominal",
    "Q81__Sow__nominal",
    "Q82__Irrigation__nominal",
    "Q84__Years_using_drip_irrigation__continuous",
]

# 10) Self-control items
self_control_features_nepal = [
    "Q80__SC_1_When_I_do_a_boring_job_I_think_about_the_less_boring_parts_of_the_job__ordinal",
    "Q81__SC_2_By_changing_my_way_of_thinking_I_am_often_able_to_change_my_feelings_about_almost_everything__ordinal",
    "Q82__SC_3_I_often_find_it_difficult_to_overcome_my_feelings_of_nervousness_and_tension_without_help__ordinal",
    "Q83__SC_4_When_I_am_feeling_depressed_I_try_to_think_about_pleasant_events__ordinal",
    "Q84__SC_5_I_cannot_help_thinking_about_mistakes_I_made__ordinal",
    "Q85__SC_6_I_usually_do_what_I_am_supposed_to_do_more_quickly_when_someone_is_pressuring_me__ordinal",
    "Q86__SC_7_When_I_am_faced_with_a_difficult_decision_I_prefer_to_postpone_it__ordinal",
    "Q87__SC_8_When_I_try_to_get_rid_of_a_bad_habit_for_example_smoking_stealing_i_first_try_to_find_out_why_i_have_the_habit__ordinal",
    "Q88__SC_9_If_I_smoked_two_packs_of_cigarettes_a_day_I_would_need_help_to_stop_smoking__ordinal",
    "Q89__SC_10_When_I_feel_down_I_try_to_act_cheerful_so_that_my_mood_will_change__ordinal",
    "Q90__SC_11_I_tend_to_postpone_unpleasant_tasks__ordinal",
    "Q91__SC_12_I_prefer_to_finish_a_job_that_I_have_to_do_before_I_start_doing_things_I_really_like__ordinal",
    "Q92__SC_13_When_I_feel_pain_I_try_not_to_think_about_it__ordinal",
    "Q93__SC_14_My_selfesteem_increases_when_I_am_able_to_overcome_a_bad_habit__ordinal",
    "Q94__SC_15_When_I_feel_that_I_am_too_impulsive_I_tell_myself_to_stop_and_think_before_I_do_anything__ordinal",
    "Q95__SC_16_Even_when_I_am_terribly_angry_at_someone_I_consider_my_actions_very_carefully__ordinal",
    "Q96__SC_17_When_I_need_to_make_a_decision_I_try_to_look_for_different_alternative__ordinal",
    "Q97__SC_18_Usually_I_first_do_the_thing_I_really_like_to_do_even_if_there_are_more_urgent_things_to_do__ordinal",
    "Q98__SC_19_When_I_am_faced_with_a_number_of_things_to_do_I_usually_plan_my_work__ordinal",
    "Q99__SC_20_When_I_am_tired_and_I_have_no_opportunity_to_sleep_I_try_not_to_think_about_it__ordinal",
]

# 11) Household, income & trust
household_features_nepal = [
    "Q100__Caste__nominal",
    "Q101__how_many_people_live_in_this_household__continuous",
    "Q105__main_sources_of_income__nominal",
    "Q106__Education_level_of_this_person_that_is_interviewed_years_of_formal_education__continuous",
    "Q107__Education_level_of_your_wife_husband_How_many_years_of_formal_education__continuous",
    "Q110__Do_you_currently_have_a_loan_if_no_zero_if_yes_amount_of_money_loaned_NPR__continuous",
    "Q111__Generally_speaking_would_you_say_that_most_people_can__ordinal",
]

household_features_senegal = [
    "Q85__Group__nominal",
    "Q87__Household_size__continuous",
    "Q88__family_main_sources_income__nominal",
    "Q89__education_level__nominal",
    "Q0__Has_Education_Yes_No__binary__1",
    "Q0__Education_years__continuous",
    "Q92__Current_loan_amount_XOF__continuous",
    "Q93__Trust_most_people__ordinal",
]

# 12) Indices & aggregates
hope_index_nepal            = ["Q0__hope_total__continuous"]
hope_index_senegal          = ["Q0__Hope_total__continuous"]
self_control_index_nepal  = ["Q0__self_control_score__continuous"]
positive_index_nepal        = ["Q0__positive_total__continuous"]
negative_index_nepal        = ["Q0__negative_total__continuous"]
avg_farming_practices_nepal = ["Q0__average_of_farming_practices__ordinal"]


Plot scatter plots with LOESS between numeric / ordinal columns and the targets and boxplots for binary and nominal features against the targets for each of the above defined groups

In [850]:
import math
from statsmodels.nonparametric.smoothers_lowess import lowess
BINARY = 1
SINGLE_DUMMY = 2
ONE_HOT = 3
MULTI_SELECT = 4


def robust_loess(x, y, **kwargs):
    fitted = lowess(endog=y, exog=x, frac=0.6667, it=3)
    plt.plot(fitted[:, 0], fitted[:, 1], **kwargs)

# 3) pairplot function

def plot_pairplots(df, expl_feats, target_feats):

    # 2) for each target, plot *only* expl → target
    for tgt in target_feats:
        tmeta = parse_feature_metadata(tgt)
        if not tmeta:
            continue
        tname = f"{tmeta['qid']}_{tmeta['name']}"

        # assemble data
        tmp = df[expl_feats].copy()
        tmp[tname] = df[tgt]
        tmp = tmp.dropna()
        if tmp.shape[0] < 2:
            continue

        # i) compute grid shape
        n = len(expl_feats)
        ncols = int(math.ceil(math.sqrt(n)))
        nrows = int(math.ceil(n / ncols))

        # ii) create subplots grid
        fig, axes = plt.subplots(nrows, ncols,
                                 figsize=(ncols * 4, nrows * 3),
                                 squeeze=False)
        axes = axes.flatten()

        # iii) plot each explanatory feature
        for i, exp in enumerate(expl_feats):
            ax = axes[i]
            meta = parse_feature_metadata(exp)
            expl_type = meta.get('type', None)

            if expl_type in ['continuous', 'discrete', 'ordinal']:
                x = tmp[exp]
                y = tmp[tname]

                # if this is ordinal (labels), convert to ordered categorical codes
                if expl_type == 'ordinal' or pd.api.types.is_categorical_dtype(x):
                    cat = pd.Categorical(x, ordered=True)
                    x_codes = cat.codes
                    # scatter & LOESS on codes
                    ax.scatter(x_codes, y, alpha=0.6)
                    fitted = lowess(endog=y, exog=x_codes, frac=0.6667, it=3)
                    ax.plot(fitted[:, 0], fitted[:, 1], color='r')
                    # relabel x-ticks with the category names
                    ax.set_xticks(range(len(cat.categories)))
                    ax.set_xticklabels(cat.categories,
                                       rotation=45, ha='right', fontsize=8)
                else:
                    # purely numeric
                    ax.scatter(x, y, alpha=0.6)
                    fitted = lowess(endog=y, exog=x, frac=0.6667, it=3)
                    ax.plot(fitted[:, 0], fitted[:, 1], color='r')

            elif expl_type in ['nominal', 'binary']:

                # detect multi-select (list entries) vs. single-select (scalars)
                is_multi = tmp[exp].apply(lambda x: isinstance(x, (list, tuple))).any()
                if is_multi:
                    # --- multi-select nominal: explode lists so each label gets its own box ---
                    df_expl = tmp[[exp, tname]].explode(exp)
                    df_expl = df_expl.dropna(subset=[exp])
                    sns.boxplot(x=df_expl[exp].astype(str),
                                y=df_expl[tname],
                                ax=ax)
                else:
                    levels = tmp[exp].astype(str).value_counts().index
                    if len(levels) <= 10:
                        sns.boxplot(x=tmp[exp].astype(str), y=tmp[tname], ax=ax)
                        #ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right', fontsize=8)
                        #make x ticks wrap like done for the xlabel below
                    else:
                        top_levels = tmp[exp].astype(str).value_counts().nlargest(10).index
                        tmp2 = tmp[tmp[exp].astype(str).isin(top_levels)]
                        sns.boxplot(x=tmp2[exp].astype(str), y=tmp2[tname], ax=ax)
                        #ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right', fontsize=8)
                        #make x ticks wrap like done for the xlabel below

                orig_labels = [lbl.get_text() for lbl in ax.get_xticklabels()]
                wrapped_labels = ["\n".join([lab[i:i+30] for i in range(0, len(lab), 30)]) for lab in orig_labels]
                ax.set_xticklabels(wrapped_labels, rotation=45, ha='right', fontsize=8)

            # per-axis label
            wrapped_label = "\n".join([exp[i:i+30] for i in range(0, len(exp), 30)])
            ax.set_xlabel(wrapped_label, fontsize=8, labelpad=8)
            ax.set_ylabel("")

        # turn off leftover axes
        for ax in axes[n:]:
            ax.set_visible(False)

        # tidy and title
        plt.tight_layout()
        plt.suptitle(f"Explanatory Features vs {tname}", y=1.02)
        plt.show()


# 4) heatmap function

def plot_heatmaps(df, expl_feats, target_feats, method='pearson'):
    # 1) subset to numeric explanatory + numeric targets
    sub = df[expl_feats + target_feats].copy()
    # 2) coerce any object/nominal into codes
    for c in sub.columns:
        if not pd.api.types.is_numeric_dtype(sub[c]):
            sub[c] = sub[c].astype("category").cat.codes
    # 3) compute corr
    corr = sub.corr(method=method).loc[expl_feats, target_feats]
    plt.figure(figsize=(len(target_feats)*0.5+3, len(expl_feats)*0.3+3))
    sns.heatmap(corr, annot=True, fmt=".2f", cmap="vlag", center=0)
    plt.title("Correlation heatmap: explanatory vs target", pad=20)
    plt.show()

def plot_group_heatmap(df, features, method='pearson'):

    sub = df[features].copy()
    # coerce nominal into codes
    for c in sub.columns:
        if not pd.api.types.is_numeric_dtype(sub[c]):
            sub[c] = sub[c].astype("category").cat.codes
    corr = sub.corr(method=method)
    sns.heatmap(corr, annot=True, fmt=".2f", cmap="viridis", center=0)
    plt.title("Correlation heatmap: group", pad=20)
    plt.show()


## Geography & Location

In [851]:
print("NEPAL DATA")
if PLOT_GRAPHS:
    plot_pairplots(nepal_df, geo_features_nepal, target_cols)


NEPAL DATA


In [852]:
print("NEPAL DATA, PEARSON CORR")
if PLOT_GRAPHS:
    plot_heatmaps(nepal_df, geo_features_nepal, target_cols)
    plot_group_heatmap(nepal_df, geo_features_nepal)

NEPAL DATA, PEARSON CORR


In [853]:
print("NEPAL DATA, SPEARMAN CORR")
if PLOT_GRAPHS:
    plot_heatmaps(nepal_df, geo_features_nepal, target_cols, 'spearman')
    plot_group_heatmap(nepal_df, geo_features_nepal, 'spearman')

NEPAL DATA, SPEARMAN CORR


In [854]:
print("SENEGAL DATA")
if PLOT_GRAPHS:
    plot_pairplots(senegal_df, geo_features_senegal, target_cols)


SENEGAL DATA


In [855]:
print("SENEGAL DATA, PEARSON CORR")
if PLOT_GRAPHS:
    plot_heatmaps(senegal_df, geo_features_senegal, target_cols)
    plot_group_heatmap(senegal_df, geo_features_senegal)

SENEGAL DATA, PEARSON CORR


In [856]:
print("SENEGAL DATA, SPEARMAN CORR")
if PLOT_GRAPHS:
    plot_heatmaps(senegal_df, geo_features_senegal, target_cols, 'spearman')
    plot_group_heatmap(senegal_df, geo_features_senegal, 'spearman')

SENEGAL DATA, SPEARMAN CORR


### Check Visually on Map

In [857]:
import geopandas as gpd
from shapely.geometry import Point
import contextily as ctx

def map_targets_on_map(
    df,
    lon_col,
    lat_col,
    target_cols,
    basemap=True,
    figsize=(8, 8),
    cmap='viridis',
    marker_size=50
):
    """
    For each target variable, plot its spatial distribution on a map.

    Parameters:
    - df: pandas.DataFrame containing data
    - lon_col, lat_col: names of longitude and latitude columns
    - target_cols: list of target variable names to map
    - basemap: whether to include a background map (default True)
    - figsize: size of the matplotlib figure
    - cmap: colormap for target values
    - marker_size: size of plotted points
    """
    # Create GeoDataFrame
    gdf = gpd.GeoDataFrame(
        df.copy(),
        geometry=gpd.points_from_xy(df[lon_col], df[lat_col]),
        crs="EPSG:4326"
    )
    # Project to Web Mercator for basemap
    gdf = gdf.to_crs(epsg=3857)

    for target in target_cols:
        fig, ax = plt.subplots(figsize=figsize)
        gdf.plot(
            ax=ax,
            column=target,
            cmap=cmap,
            legend=True,
            markersize=marker_size
        )
        if basemap:
            # Attempt Stamen terrain; fallback to OSM if unavailable
            try:
                provider = ctx.providers.Stamen.Terrain
            except Exception:
                provider = ctx.providers.OpenStreetMap.Mapnik
            ctx.add_basemap(ax, source=provider)
        ax.set_axis_off()
        ax.set_title(f"Spatial distribution of {target}")
        plt.tight_layout()
        plt.show()

In [858]:
if PLOT_GRAPHS:
    map_targets_on_map(nepal_df, 'Q1__Longitude__continuous', 'Q1__Latitude__continuous', target_cols)

In [859]:
if PLOT_GRAPHS:
    map_targets_on_map(senegal_df, 'Q1__Longitude__continuous', 'Q1__Latitude__continuous', target_cols)

## Demographic Data

In [860]:
if PLOT_GRAPHS:
    plot_pairplots(nepal_df, demo_features_nepal, target_cols)

In [861]:
if PLOT_GRAPHS:
    plot_pairplots(senegal_df, demo_features_senegal, target_cols)


## Hope Data

In [862]:
print("NEPAL DATA")
if PLOT_GRAPHS:
    plot_pairplots(nepal_df, hope_features_nepal, target_cols)


NEPAL DATA


In [863]:
print("NEPAL DATA, PEARSON CORR")
if PLOT_GRAPHS:
    plot_heatmaps(nepal_df, hope_features_nepal, target_cols)
    plot_group_heatmap(nepal_df, hope_features_nepal)

NEPAL DATA, PEARSON CORR


In [864]:
print("NEPAL DATA, SPEARMAN CORR")
if PLOT_GRAPHS:
    plot_heatmaps(nepal_df, hope_features_nepal, target_cols, method='spearman')
    plot_group_heatmap(nepal_df, hope_features_nepal, method='spearman')

NEPAL DATA, SPEARMAN CORR


In [865]:
print("SENEGAL DATA")
if PLOT_GRAPHS:
    plot_pairplots(senegal_df, hope_features_senegal, target_cols)


SENEGAL DATA


In [866]:
print("SENEGAL DATA, PEARSON CORR")
if PLOT_GRAPHS:
    plot_heatmaps(senegal_df, hope_features_senegal, target_cols)
    plot_group_heatmap(senegal_df, hope_features_senegal)

SENEGAL DATA, PEARSON CORR


In [867]:
print("SENEGAL DATA, SPEARMAN CORR")
if PLOT_GRAPHS:
    plot_heatmaps(senegal_df, hope_features_senegal, target_cols, method='spearman')
    plot_group_heatmap(senegal_df, hope_features_senegal, method='spearman')

SENEGAL DATA, SPEARMAN CORR


## Crop & Fertilizer Data

In [868]:
print("NEPAL DATA")
if PLOT_GRAPHS:
    plot_pairplots(nepal_df, crop_fertilizer_features_nepal, target_cols)


NEPAL DATA


In [869]:
print("NEPAL DATA, PEARSON CORR")
if PLOT_GRAPHS:
    plot_heatmaps(nepal_df, crop_fertilizer_features_nepal, target_cols)
    plot_group_heatmap(nepal_df, crop_fertilizer_features_nepal)

NEPAL DATA, PEARSON CORR


In [870]:
print("NEPAL DATA, SPEARMAN CORR")
if PLOT_GRAPHS:
    plot_heatmaps(nepal_df, crop_fertilizer_features_nepal, target_cols, 'spearman')
    plot_group_heatmap(nepal_df, crop_fertilizer_features_nepal, 'spearman')

NEPAL DATA, SPEARMAN CORR


In [871]:
print("SENEGAL DATA")
if PLOT_GRAPHS:
    plot_pairplots(senegal_df, crop_fertilizer_features_senegal, target_cols)


SENEGAL DATA


In [872]:
print("SENEGAL DATA, PEARSON CORR")
if PLOT_GRAPHS:
    plot_heatmaps(senegal_df, crop_fertilizer_features_senegal, target_cols)
    plot_group_heatmap(senegal_df, crop_fertilizer_features_senegal)

SENEGAL DATA, PEARSON CORR


In [873]:
print("SENEGAL DATA, SPEARMAN CORR")
if PLOT_GRAPHS:
    plot_heatmaps(senegal_df, crop_fertilizer_features_senegal, target_cols, method='spearman')
    plot_group_heatmap(senegal_df, crop_fertilizer_features_senegal, method='spearman')

SENEGAL DATA, SPEARMAN CORR


## Pest Control & Machinery & Weeding

In [874]:
print("NEPAL DATA")
if PLOT_GRAPHS:
    plot_pairplots(nepal_df, plot_practice_features_nepal, target_cols)


NEPAL DATA


In [875]:
print("NEPAL DATA, PEARSON CORR")
if PLOT_GRAPHS:
    plot_heatmaps(nepal_df, plot_practice_features_nepal, target_cols)
    plot_group_heatmap(nepal_df, plot_practice_features_nepal)

NEPAL DATA, PEARSON CORR


In [876]:
print("NEPAL DATA, SPEARMAN CORR")
if PLOT_GRAPHS:
    plot_heatmaps(nepal_df, plot_practice_features_nepal, target_cols, method='spearman')
    plot_group_heatmap(nepal_df, plot_practice_features_nepal, method='spearman')

NEPAL DATA, SPEARMAN CORR


In [877]:
print("SENEGAL DATA")
if PLOT_GRAPHS:
    plot_pairplots(senegal_df, plot_practice_features_senegal, target_cols)


SENEGAL DATA


In [878]:
print("SENEGAL DATA, PEARSON CORR")
if PLOT_GRAPHS:
    plot_heatmaps(senegal_df, plot_practice_features_senegal, target_cols)
    plot_group_heatmap(senegal_df, plot_practice_features_senegal)

SENEGAL DATA, PEARSON CORR


In [879]:
print("SENEGAL DATA, SPEARMAN CORR")
if PLOT_GRAPHS:
    plot_heatmaps(senegal_df, plot_practice_features_senegal, target_cols, method='spearman')
    plot_group_heatmap(senegal_df, plot_practice_features_senegal, method='spearman')

SENEGAL DATA, SPEARMAN CORR


In [880]:
print("SENEGAL DATA")
if PLOT_GRAPHS:
    plot_pairplots(senegal_df, soil_features_senegal, target_cols)


SENEGAL DATA


In [881]:
print("SENEGAL DATA, PEARSON CORR")
if PLOT_GRAPHS:
    plot_heatmaps(senegal_df, soil_features_senegal, target_cols)
    plot_group_heatmap(senegal_df, soil_features_senegal)

SENEGAL DATA, PEARSON CORR


In [882]:
print("SENEGAL DATA, SPEARMAN CORR")
if PLOT_GRAPHS:
    plot_heatmaps(senegal_df, soil_features_senegal, target_cols, method='spearman')
    plot_group_heatmap(senegal_df, soil_features_senegal, method='spearman')

SENEGAL DATA, SPEARMAN CORR


## Plowing, Sowing & Irrigation

In [883]:
print("NEPAL DATA, PEARSON CORR")
if PLOT_GRAPHS:
    plot_heatmaps(nepal_df, plow_irrigation_features_nepal, target_cols)
    plot_group_heatmap(nepal_df, plow_irrigation_features_nepal)

NEPAL DATA, PEARSON CORR


In [884]:
print("NEPAL DATA, SPEARMAN CORR")
if PLOT_GRAPHS:
    plot_heatmaps(nepal_df, plow_irrigation_features_nepal, target_cols, method='spearman')
    plot_group_heatmap(nepal_df, plow_irrigation_features_nepal, method='spearman')

NEPAL DATA, SPEARMAN CORR


In [885]:
print("SENEGAL DATA")
if PLOT_GRAPHS:
    plot_pairplots(senegal_df, plow_irrigation_features_senegal, target_cols)


SENEGAL DATA


In [886]:
print("SENEGAL DATA, PEARSON CORR")
if PLOT_GRAPHS:
    plot_heatmaps(senegal_df, plow_irrigation_features_senegal, target_cols)
    plot_group_heatmap(senegal_df, plow_irrigation_features_senegal)

SENEGAL DATA, PEARSON CORR


In [887]:
print("SENEGAL DATA, SPEARMAN CORR")
if PLOT_GRAPHS:
    plot_heatmaps(senegal_df, plow_irrigation_features_senegal, target_cols, method='spearman')
    plot_group_heatmap(senegal_df, plow_irrigation_features_senegal, method='spearman')

SENEGAL DATA, SPEARMAN CORR


## Self-control Items

In [888]:
print("NEPAL DATA")
if PLOT_GRAPHS:
    plot_pairplots(nepal_df, self_control_features_nepal, target_cols)


NEPAL DATA


In [889]:
print("SENEGAL DATA, PEARSON CORR")
if PLOT_GRAPHS:
    plot_heatmaps(senegal_df, plow_irrigation_features_senegal, target_cols)
    plot_group_heatmap(senegal_df, plow_irrigation_features_senegal)

SENEGAL DATA, PEARSON CORR


In [890]:
print("SENEGAL DATA, SPEARMAN CORR")
if PLOT_GRAPHS:
    plot_heatmaps(senegal_df, plow_irrigation_features_senegal, target_cols, method='spearman')
    plot_group_heatmap(senegal_df, plow_irrigation_features_senegal, method='spearman')

SENEGAL DATA, SPEARMAN CORR


## Household & Income & Trust

In [891]:
print("NEPAL DATA")
if PLOT_GRAPHS:
    plot_pairplots(nepal_df, household_features_nepal, target_cols)


NEPAL DATA


In [892]:
print("NEPAL DATA, PEARSON CORR")
if PLOT_GRAPHS:
    plot_heatmaps(nepal_df, household_features_nepal, target_cols)
    plot_group_heatmap(nepal_df, household_features_nepal)

NEPAL DATA, PEARSON CORR


In [893]:
print("NEPAL DATA, SPEARMAN CORR")
if PLOT_GRAPHS:
    plot_heatmaps(nepal_df, household_features_nepal, target_cols, method='spearman')
    plot_group_heatmap(nepal_df, household_features_nepal, method='spearman')

NEPAL DATA, SPEARMAN CORR


In [894]:
print("SENEGAL DATA")
if PLOT_GRAPHS:
    plot_pairplots(senegal_df, household_features_senegal, target_cols)


SENEGAL DATA


In [895]:
print("SENEGAL DATA, PEARSON CORR")
if PLOT_GRAPHS:
    plot_heatmaps(senegal_df, household_features_senegal, target_cols)
    plot_group_heatmap(senegal_df, household_features_senegal)

SENEGAL DATA, PEARSON CORR


In [896]:
print("SENEGAL DATA, SPEARMAN CORR")
if PLOT_GRAPHS:
    plot_heatmaps(senegal_df, household_features_senegal, target_cols, method='spearman')
    plot_group_heatmap(senegal_df, household_features_senegal, method='spearman')

SENEGAL DATA, SPEARMAN CORR


In [897]:
print("NEPAL DATA")
if PLOT_GRAPHS:
    plot_pairplots(nepal_df, self_control_index_nepal, target_cols)

NEPAL DATA


In [898]:
print("NEPAL DATA, PEARSON CORR")
if PLOT_GRAPHS:
    plot_heatmaps(nepal_df, self_control_index_nepal, target_cols)


NEPAL DATA, PEARSON CORR


In [899]:
print("NEPAL DATA, SPEARMAN CORR")
if PLOT_GRAPHS:
    plot_heatmaps(nepal_df, self_control_index_nepal, target_cols, method='spearman')


NEPAL DATA, SPEARMAN CORR


In [900]:

print("NEPAL DATA")
if PLOT_GRAPHS:
    plot_pairplots(nepal_df, hope_index_nepal, target_cols)

NEPAL DATA


In [901]:
print("NEPAL DATA, PEARSON CORR")
if PLOT_GRAPHS:
    plot_heatmaps(nepal_df, hope_index_nepal, target_cols)

NEPAL DATA, PEARSON CORR


In [902]:
print("NEPAL DATA, SPEARMAN CORR")
if PLOT_GRAPHS:
    plot_heatmaps(nepal_df, hope_index_nepal, target_cols, method='spearman')

NEPAL DATA, SPEARMAN CORR


In [903]:

print("SENEGAL DATA")
if PLOT_GRAPHS:
    plot_pairplots(senegal_df, hope_index_senegal, target_cols)

SENEGAL DATA


In [904]:
print("SENEGAL DATA, PEARSON CORR")
if PLOT_GRAPHS:
    plot_heatmaps(senegal_df, hope_index_senegal, target_cols)

SENEGAL DATA, PEARSON CORR


In [905]:
print("SENEGAL DATA, SPEARMAN CORR")
if PLOT_GRAPHS:
    plot_heatmaps(senegal_df, hope_index_senegal, target_cols, method='spearman')

SENEGAL DATA, SPEARMAN CORR


In [906]:
print("NEPAL DATA")
if PLOT_GRAPHS:
    plot_pairplots(nepal_df, positive_index_nepal, target_cols)

NEPAL DATA


In [907]:
print("NEPAL DATA, PEARSON CORR")
if PLOT_GRAPHS:
    plot_heatmaps(nepal_df, positive_index_nepal, target_cols)

NEPAL DATA, PEARSON CORR


In [908]:
print("NEPAL DATA, SPEARMAN CORR")
if PLOT_GRAPHS:
    plot_heatmaps(nepal_df, positive_index_nepal, target_cols, method='spearman')

NEPAL DATA, SPEARMAN CORR


In [909]:
print("NEPAL DATA")
if PLOT_GRAPHS:
    plot_pairplots(nepal_df, negative_index_nepal, target_cols)

NEPAL DATA


In [910]:
print("NEPAL DATA, PEARSON CORR")
if PLOT_GRAPHS:
    plot_heatmaps(nepal_df, negative_index_nepal, target_cols)

NEPAL DATA, PEARSON CORR


In [911]:
print("NEPAL DATA, SPEARMAN CORR")
if PLOT_GRAPHS:
    plot_heatmaps(nepal_df, negative_index_nepal, target_cols, method='spearman')

NEPAL DATA, SPEARMAN CORR


In [912]:
print("NEPAL DATA")
if PLOT_GRAPHS:
    plot_pairplots(nepal_df, avg_farming_practices_nepal, target_cols)

NEPAL DATA


In [913]:
print("NEPAL DATA, PEARSON CORR")
if PLOT_GRAPHS:
    plot_heatmaps(nepal_df, avg_farming_practices_nepal, target_cols)

NEPAL DATA, PEARSON CORR


In [914]:
print("NEPAL DATA, SPEARMAN CORR")
if PLOT_GRAPHS:
    plot_heatmaps(nepal_df, avg_farming_practices_nepal, target_cols, method='spearman')

NEPAL DATA, SPEARMAN CORR


In [915]:
print("NEPAL DATA")
if PLOT_GRAPHS:
    plot_pairplots(nepal_df, feelings_features_nepal, target_cols)

NEPAL DATA


In [916]:
print("NEPAL DATA, PEARSON CORR")
if PLOT_GRAPHS:
    plot_heatmaps(nepal_df, feelings_features_nepal, target_cols)

NEPAL DATA, PEARSON CORR


In [917]:
print("NEPAL DATA, SPEARMAN CORR")
if PLOT_GRAPHS:
    plot_heatmaps(nepal_df, feelings_features_nepal, target_cols, method='spearman')

NEPAL DATA, SPEARMAN CORR


In [918]:
print("SENEGAL DATA")
if PLOT_GRAPHS:
    plot_pairplots(senegal_df, character_features_senegal, target_cols)

SENEGAL DATA


## Productivity Metrics Heatmap

In [919]:
if PLOT_GRAPHS:
    plot_group_heatmap(nepal_df, target_cols)

In [920]:
if PLOT_GRAPHS:
    plot_group_heatmap(senegal_df, target_cols)

# Second Clean

Change phone number column to just binary yes no if he has a phone number

In [921]:

nepal_df['Q4__PhoneNumberAndIsZeroIfNone__nominal'] = (nepal_df['Q4__PhoneNumberAndIsZeroIfNone__nominal'] == 0).astype(int)
senegal_df['Q4__Phone_Number__nominal'] = (senegal_df['Q4__Phone_Number__nominal'] == 0).astype(int)

In [922]:
nepal_df.rename(columns={'Q4__PhoneNumberAndIsZeroIfNone__nominal' : 'Q4__HasPhoneNumber__binary__1'}, inplace=True)
senegal_df.rename(columns={'Q4__Phone_Number__nominal' : 'Q4__HasPhoneNumber__binary__1'}, inplace=True)


Scale all numeric columns and one hot encode all categorical columns, and deal with missing values in categorical columns by adding a 'missing' category

In [923]:
nepal_df[hope_features_nepal]

,Q17__Hope_1_I_Can_Think_Of_Many_Ways_To_Get_Out_Of_A_Jam__ordinal,Q18__Hope_2_I_energetically_pursue_my_goals__ordinal,Q19__Hope_3_I_feel_tired_most_of_the_time__ordinal,Q20__Hope_4_There_are_lots_of_ways_around_any_problem__ordinal,Q21__Hope_5_I_am_easily_downed_being_at_a_low_position_brought_down_in_an_argument__ordinal,Q22__Hope_6_I_can_think_of_many_ways_to_get_the_things_in_life_that_are_important_to_me__ordinal,Q23__Hope_7_I_worry_about_my_health__ordinal,Q24__Hope_8_Even_when_others_get_discouraged_I_know_I_can_find_a_way_to_solve_the_problem__ordinal,Q25__Hope_9_My_past_experiences_have_prepared_me_well_for_my_future__ordinal,Q26__Hope_10_I_have_been_pretty_successful_in_life__ordinal,Q27__Hope_11_I_usually_find_myself_worrying_about_something__ordinal,Q28__Hope_12_I_meet_the_goals_that_I_set_for_myself__ordinal
0,3,5,3,6,6,6,8,7,6,6,6,4
1,6,7,2,6,1,7,8,5,7,6,5,6
2,5,7,2,6,5,7,8,5,7,5,7,6
3,6,6,5,7,4,6,6,7,8,6,4,6
4,5,6,3,6,4,7,8,6,5,5,6,5
...,...,...,...,...,...,...,...,...,...,...,...,...
239,8,7,2,8,1,7,5,5,8,6,3,6
240,7,8,5,7,2,6,8,5,8,8,5,7
241,8,8,4,7,5,7,8,6,8,8,4,7
242,7,8,5,7,4,7,8,7,7,7,5,7


In [924]:
from sklearn.preprocessing import StandardScaler
import re

def scale_numeric_columns(df, scale_ordinal=True):
    df = df.copy()
    # Get non-categorical numeric columns
    cols_to_scale = [col for col in df.columns if parse_feature_metadata(col)["type"] in ['continuous', 'discrete', 'ordinal' if scale_ordinal else None]]

    for col in cols_to_scale:
        median = df[col].median()
        # this returns a new Series (up-cast to float64) and assigns it back
        df[col] = df[col].fillna(median)
    
    scaler = StandardScaler()
    df[cols_to_scale] = scaler.fit_transform(df[cols_to_scale])

    return df

def deal_with_missing_in_cat_cols(df):

    nom_cols = [col for col in df.columns if parse_feature_metadata(col)["type"] in ['nominal']]
    ord_cols = [col for col in df.columns if parse_feature_metadata(col)["type"] in ['ordinal']]

    # deal with missing by adding a 'Missing' category
    for col in nom_cols:
        df[col] = df[col].astype('category').cat.add_categories('Missing')
        df[col] = df[col].fillna('Missing')
    for col in ord_cols:
        # Find the mode of the 'category' column
        mode_value = df[col].astype('category').mode()[0]
        # Fill missing values with the mode
        df[col].fillna(mode_value, inplace=True)

def one_hot_encode_columns(df):
    df = df.copy()
    cols_to_dummies = []
    cols_to_dummies_prefixes = []
    for col in df.columns:
        meta = parse_feature_metadata(col)
        if meta["type"] in ['ordinal', 'nominal']:
            # add it to columns to one hot encode along with its question number
            cols_to_dummies_prefixes.append(f"{meta['qid']}__{meta['name']}")
            cols_to_dummies.append((col, meta["type"]))
    
    col_names_to_dummies, _ = zip(*cols_to_dummies) 
    col_names_to_dummies = list(col_names_to_dummies)
    # applying prefix to dummies according to our feature name format
    dummies = pd.get_dummies(df[col_names_to_dummies], prefix_sep='_', prefix=cols_to_dummies_prefixes, columns=col_names_to_dummies, dtype=int)
    
    def reformat(col):
        qid, name = col.split("__")
        return f"{qid}__{re.sub(r'[^A-Za-z0-9_]', '', name)}"

    dummies = dummies.rename(columns=reformat)

    # applying postfix to dummies according to our feature name format
    rename_map = {}
    for orig, type in cols_to_dummies:
        # find all dummy cols that start with "<orig's qid>-<orig's name>"
        group = [c for c in dummies.columns if c.startswith(f"{parse_feature_metadata(orig)['qid']}__{parse_feature_metadata(orig)['name']}")]
        # enumerate them 1,2,3…
        for i, col_name in enumerate(group, start=1):
            rename_map[col_name] = f"{col_name}__binary_{type}__{i}"
            
    dummies = dummies.rename(columns=rename_map)
    df_other = df.drop(columns=col_names_to_dummies)
    final_df = pd.concat([df_other, dummies], axis=1)
    
    return final_df



In [925]:
def get_nominals(df):
    cols = []
    for col in df.columns:
        if parse_feature_metadata(col)['type'] == 'nominal':
            cols.append(col)
    return cols

In [926]:
def convert_ordinal_to_numeric(df):
    for col in df.columns:
        if parse_feature_metadata(col)['type'] == 'ordinal':
            df[col] = df[col].apply(lambda x: int(x[-1]) if isinstance(x, (str)) else x)


In [927]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

def reduce_to_index(df: pd.DataFrame,
                    cols: list[str],
                    new_col_name: str = "PC1") -> pd.DataFrame:
    """
    Runs PCA on the specified columns of df and replaces them with the first component.
    
    Parameters
    ----------
    df : pd.DataFrame
        Your input DataFrame.
    cols : list of str
        List of exactly four column names in df to be reduced.
    new_col_name : str, default="PC1"
        Name of the new column for the first principal component.
    
    Returns
    -------
    pd.DataFrame
        A copy of df where the four columns in `cols` are dropped and replaced
        by `new_col_name`, containing the first principal component scores.
    """
    
    # 1. Standardize the selected columns
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(df[cols])
    
    # 2. Fit PCA and transform
    pca = PCA(n_components=4).fit(X_scaled)
    scores = pca.transform(X_scaled)

    # 3. Build the variance-weighted composite index of the first two PCs
    var_props = pca.explained_variance_ratio_[:2]
    weights = var_props / var_props.sum()
    index = (scores[:, :2] * weights).sum(1)

    # 4. Construct the new DataFrame
    df_out = df.copy()
    df_out[new_col_name] = index
    df_out = df_out.drop(columns=cols)
    
    return df_out

In [928]:
target_cols = ['Q0__target1_yearly_income_from_agr_per_land_SQM__continuous',
               'Q0__target2_yearly_income_from_agr_USD__continuous',
               'Q0__target3_veg_per_area__continuous',
               'Q0__target4_self_farming_perception__ordinal']

Supervisor asked us to not use specific psychological indicators in our final method

In [929]:
NEPAL_HOPE_QUESTIONS_RANGE = list(range(17,29))
NEPAL_FEELINGS_QUESTIONS_RANGE = list(range(30, 50))
NEPAL_SELF_CONTROL_QUESTIONS_RANGE = list(range(80, 100))
NEPAL_PSYCH_QUESTION_RANGE = NEPAL_HOPE_QUESTIONS_RANGE \
                            + NEPAL_FEELINGS_QUESTIONS_RANGE \
                            + NEPAL_SELF_CONTROL_QUESTIONS_RANGE

SENEGAL_HOPE_QUESTIONS_RANGE = list(range(11,23))
SENEGAL_CHARACTER_STRENGTH_QUESTIONS_RANGE = list(range(24, 60))
SENEGAL_PSYCH_QUESTION_RANGE = SENEGAL_HOPE_QUESTIONS_RANGE \
                                + SENEGAL_CHARACTER_STRENGTH_QUESTIONS_RANGE

NEPAL_ADDITIONAL = ["Q1__Latitude__continuous", "Q1__Longitude__continuous", "Q1__Accuracy__continuous", "Q50__How_much_land_that_is_yours_do_you_cultivate_bigha__continuous",
                    "Q51__How_much_land_that_is_rented_or_leased_do_you_cultivate_bigha__continuous", "Q62__How_much_VEGETABLES_do_you_harvest_per_year_from_this_plot_kilograms__continuous",
                    "Q108__What_is_your_households_yearly_income_from_agriculture_NPR__continuous", "Q52__On_how_much_land_do_you_grow_vegetables_bigha__continuous",
                    "Q57__If_you_self_prepare_seedlings_how_Nursery__binary__1", "Q55__For_Vegetables_do_you_use_improved_and_or_variety_seeds_yes__binary__1",
                    "Q10__SexMale__binary__1", "Q0__positive_total__continuous", "Q0__negative_total__continuous"]
SENEGAL_ADDITIONAL = ["Q1__Latitude__continuous", "Q1__Longitude__continuous", "Q1__Accuracy__continuous", "Q60__Land_owned_cultivated_ha__continuous", "Q61__Land_rented_cultivated_ha__continuous",
                      "Q71__VEG_harvest_per_year_kg__continuous", "Q90__Yearly_income_agriculture_XOF__continuous", "Q69__Use_pesticide_or_herbicide__binary__1", "Q0__Distance_Thies_KM__continuous",
                      "Q0__Distance_Dakar_KM__continuous", "Q62__Land_grow_vegetables_ha__continuous"]

def drop_ordinal_psych_columns(df, country):
    for col in df.columns:
        qid = int(parse_feature_metadata(col)["qid"].replace('Q', ''))
        if country == "NEPAL" and (qid in NEPAL_PSYCH_QUESTION_RANGE or col in NEPAL_ADDITIONAL):
            df.drop(col, axis=1, inplace=True)
        if country == "SENEGAL" and (qid in SENEGAL_PSYCH_QUESTION_RANGE or col in SENEGAL_ADDITIONAL):
            df.drop(col, axis=1, inplace=True)


In [930]:
convert_ordinal_to_numeric(nepal_df)
deal_with_missing_in_cat_cols(nepal_df)
nepal_df_to_save = reduce_to_index(nepal_df, target_cols, new_col_name='Q0__AGR_PROD__continuous')
# No need to scale when using correlation matrix
#nepal_df_to_save = scale_numeric_columns(nepal_df_to_save, scale_ordinal=False)
drop_ordinal_psych_columns(nepal_df_to_save, "NEPAL")
nepal_df_to_save.to_excel("nepal_dataframe_FA.xlsx")
nepal_nominals = get_nominals(nepal_df_to_save)
nepal_df_nominals = one_hot_encode_columns(nepal_df_to_save[nepal_nominals])
nepal_df_to_save = nepal_df_to_save.drop(columns=nepal_nominals)
nepal_df_to_save = pd.concat((nepal_df_to_save.reset_index(drop=True), nepal_df_nominals.reset_index(drop=True)), axis=1)
nepal_df_to_save.to_excel("nepal_dataframe_SEM.xlsx")

C:\Users\nadav\AppData\Local\Temp\ipykernel_30844\677600633.py:32: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(mode_value, inplace=True)


In [931]:
nepal_df = one_hot_encode_columns(nepal_df)
nepal_df = scale_numeric_columns(nepal_df, scale_ordinal=True)


In [932]:
convert_ordinal_to_numeric(senegal_df)
deal_with_missing_in_cat_cols(senegal_df)
senegal_df_to_save = reduce_to_index(senegal_df, target_cols, new_col_name='Q0__AGR_PROD__continuous')
# No need to scale when using correlation matrix
#senegal_df_to_save = scale_numeric_columns(senegal_df_to_save, scale_ordinal=False)
drop_ordinal_psych_columns(senegal_df_to_save, "SENEGAL")
senegal_df_to_save.to_excel("senegal_dataframe_FA.xlsx")
senegal_nominals = get_nominals(senegal_df_to_save)
senegal_df_nominals = one_hot_encode_columns(senegal_df_to_save[senegal_nominals])
senegal_df_to_save = senegal_df_to_save.drop(columns=senegal_nominals)
senegal_df_to_save = pd.concat((senegal_df_to_save.reset_index(drop=True), senegal_df_nominals.reset_index(drop=True)), axis=1)
senegal_df_to_save.to_excel("senegal_dataframe_SEM.xlsx")

C:\Users\nadav\AppData\Local\Temp\ipykernel_30844\677600633.py:32: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(mode_value, inplace=True)


In [933]:
senegal_df = one_hot_encode_columns(senegal_df)
senegal_df = scale_numeric_columns(senegal_df, scale_ordinal=True)


# Multivariate Analysis

In [934]:
def convert_group_to_dummies(df, group : list):
    dummy_group = []
    for orig in group:
        meta = parse_feature_metadata(orig)
        if meta['type'] not in ['ordinal', 'nominal']:
            if orig in df.columns:
                dummy_group.append(orig)    
        else:    
            for col in df.columns:
                if col.startswith(f"{meta['qid']}__{meta['name']}"):
                    dummy_group.append(col)
                
    return dummy_group

In [935]:
groups = {'Geographic Features': (geo_features_nepal, geo_features_senegal),
          'Demographic Features': (demo_features_nepal, demo_features_senegal),
          'Hope Features': (hope_features_nepal, hope_features_senegal),
          'Crop & Fertilizer Features':(crop_fertilizer_features_nepal, crop_fertilizer_features_senegal),
          'Plot Practice Features': (plot_practice_features_nepal, plot_practice_features_senegal),
          'Plow & Irrigation': (plow_irrigation_features_nepal, plow_irrigation_features_senegal),
          'Self-control Features': (self_control_features_nepal, None),
          'Household Features': (household_features_nepal, household_features_senegal)}

In [936]:
from skbio.stats.distance import permanova, DistanceMatrix
from sklearn.preprocessing import OneHotEncoder
from gower import gower_matrix          # pip install gower
# X_enc = result of your pipeline's fit_transform

def transform(df, X_features, Y_features):
    # — X side: impute, drop zero‐var, cast to float —
    X_df    = df[X_features].copy().fillna(df[X_features].median())
    mask_x  = X_df.std(ddof=0) > 0
    X_enc   = X_df.loc[:, mask_x].astype(float).values

    # — Y side: as before —
    Y_df    = df[Y_features].copy().fillna(df[Y_features].median())
    stds    = Y_df.std(ddof=0)
    keep    = stds > 0
    Y_enc   = (Y_df.loc[:, keep] - Y_df.loc[:, keep].mean()) / stds[keep]

    return X_enc, Y_enc


def convert_to_dist_matrices(X_enc, Y_enc):
    D = gower_matrix(X_enc)
    print("shape of D:", D.shape)
    print("any NaNs in D?", np.isnan(D).any())
    print("is D symmetric?", np.allclose(D, D.T, equal_nan=False))
    DX = DistanceMatrix(gower_matrix(X_enc))
    DY = DistanceMatrix(gower_matrix(Y_enc))
    return DX, DY


In [937]:
from skbio.stats.distance import mantel

def mantel_test(DX, DY):
    mantel_r, mantel_p, _ = mantel(DX, DY, method='spearman', permutations=999)
    print(f"Mantel ρ = {mantel_r:.3f},  p = {mantel_p:.4f}")


In [938]:
import dcor

def distance_covariance_test(X_enc, Y_enc):
    dcov_p, dcov_stat = dcor.independence.distance_covariance_test(
        X_enc, Y_enc, num_resamples=999)

    print(f"dCov statistic = {dcov_stat:.4f},  p = {dcov_p:.4f}\n")



In [939]:
nepal_df.isna().any().any()

np.False_

In [940]:
Y_features = convert_group_to_dummies(nepal_df, target_cols)
for key, value in groups.items():
    print(f"Mantel and dCov tests on the {key}:")
    X_feats = convert_group_to_dummies(nepal_df, value[0])
    X_enc, Y_enc = transform(nepal_df, X_feats, Y_features)

    # no need to re-drop X here, we do both in convert_to_dist_matrices
    DX, DY = convert_to_dist_matrices(X_enc, Y_enc)

    mantel_test(DX, DY)
    distance_covariance_test(X_enc, Y_enc)   # still runs on raw arrays

Mantel and dCov tests on the Geographic Features:
shape of D: (244, 244)
any NaNs in D? False
is D symmetric? True
Mantel ρ = 0.001,  p = 0.9800
dCov statistic = 17.0628,  p = 0.0010

Mantel and dCov tests on the Demographic Features:
shape of D: (244, 244)
any NaNs in D? False
is D symmetric? True
Mantel ρ = 0.035,  p = 0.3820
dCov statistic = 1.6352,  p = 0.1000

Mantel and dCov tests on the Hope Features:
shape of D: (244, 244)
any NaNs in D? False
is D symmetric? True
Mantel ρ = 0.069,  p = 0.0010
dCov statistic = 23.7343,  p = 0.0010

Mantel and dCov tests on the Crop & Fertilizer Features:
shape of D: (244, 244)
any NaNs in D? False
is D symmetric? True
Mantel ρ = 0.417,  p = 0.0010
dCov statistic = 64.5079,  p = 0.0010

Mantel and dCov tests on the Plot Practice Features:
shape of D: (244, 244)
any NaNs in D? False
is D symmetric? True
Mantel ρ = 0.293,  p = 0.0010
dCov statistic = 45.0307,  p = 0.0010

Mantel and dCov tests on the Plow & Irrigation:
shape of D: (244, 244)
any N

In [941]:
Y_features = convert_group_to_dummies(senegal_df, target_cols)
for key in groups.keys():
    if 'Self-control' in key:
        continue
    print(f"Mantel and dCov tests on the {key}:")
    X_features = convert_group_to_dummies(senegal_df, groups[key][1])
    X_enc, Y_enc = transform(senegal_df, X_features, Y_features)
    DX, DY = convert_to_dist_matrices(X_enc, Y_enc)
    mantel_test(DX, DY)
    distance_covariance_test(X_enc, Y_enc)

Mantel and dCov tests on the Geographic Features:
shape of D: (274, 274)
any NaNs in D? False
is D symmetric? True
Mantel ρ = 0.069,  p = 0.0010
dCov statistic = 44.3139,  p = 0.0010

Mantel and dCov tests on the Demographic Features:
shape of D: (274, 274)
any NaNs in D? False
is D symmetric? True
Mantel ρ = 0.005,  p = 0.7890
dCov statistic = 1.6847,  p = 0.0050

Mantel and dCov tests on the Hope Features:
shape of D: (274, 274)
any NaNs in D? False
is D symmetric? True
Mantel ρ = 0.150,  p = 0.0010
dCov statistic = 30.6487,  p = 0.0010

Mantel and dCov tests on the Crop & Fertilizer Features:
shape of D: (274, 274)
any NaNs in D? False
is D symmetric? True
Mantel ρ = 0.016,  p = 0.3820
dCov statistic = 7.4604,  p = 0.0010

Mantel and dCov tests on the Plot Practice Features:
shape of D: (274, 274)
any NaNs in D? False
is D symmetric? True
Mantel ρ = 0.443,  p = 0.0010
dCov statistic = 86.3505,  p = 0.0010

Mantel and dCov tests on the Plow & Irrigation:
shape of D: (274, 274)
any Na

# Dimensionality Reduction Analysis

In [942]:
from sklearn.decomposition import PCA

In [943]:
def plot_cumulative_explained_variance_for_group_for_pca(df, group_name, group_features):
    # Perform PCA with additional components
    pca_full = PCA(n_components=min(len(group_features),10))  # Analyze up to 10 components
    pca_full_result = pca_full.fit_transform(df[group_features])
    
    # Cumulative explained variance
    cumulative_variance = pca_full.explained_variance_ratio_.cumsum()
    
    # Plot cumulative explained variance
    plt.figure(figsize=(10, 6))
    plt.plot(range(1, len(cumulative_variance) + 1), cumulative_variance, marker='o')
    plt.title("Cumulative Explained Variance by PCA Components")
    plt.xlabel("Number of Principal Components")
    plt.ylabel("Cumulative Explained Variance")
    plt.grid(True)
    plt.show()

In [944]:
print("NEPAL")
if PLOT_GRAPHS:
    for key, value in groups.items():
        for target in target_cols[:-1]:
            print(f"PCA of {key}, and then coloring by {target}")
            X_features = convert_group_to_dummies(nepal_df, value[0])

            pca_2d = PCA(n_components=2)
            pca_2d_result = pca_2d.fit_transform(nepal_df[X_features])
            #kmeans = KMeans(n_clusters=groups_k[key], random_state=42)
            #kmeans_result = kmeans.fit_predict(nepal_df[X_features])
            #kmeans_labels = kmeans.labels_
            plt.figure(figsize=(10, 8))
            plt.scatter(pca_2d_result[:, 0], pca_2d_result[:, 1], c=nepal_df[target], cmap='viridis', alpha=0.6)
            plt.title(f"PCA of {key}, and then coloring by {target}")
            plt.xlabel("Principal Component 1")
            plt.ylabel("Principal Component 2")
            name = parse_feature_metadata(target)['name']
            plt.colorbar(label=f"{name}")
            plt.grid(True)
            plt.show()
            
            #for target in target_features:
                
            #    print_cluster_info_for_target(nepal_df, target, kmeans_labels)

        plot_cumulative_explained_variance_for_group_for_pca(nepal_df, key, X_features)

NEPAL


In [945]:
print("SENEGAL")
if PLOT_GRAPHS:
    for key in groups.keys():
        if 'Demographic' in key or 'Self-control' in key:
                continue
        for target in target_cols[:-1]:
            
            print(f"PCA of {key}, and then coloring by {target}") 
            X_features = convert_group_to_dummies(senegal_df, groups[key][1])

            pca_2d = PCA(n_components=2)
            pca_2d_result = pca_2d.fit_transform(senegal_df[X_features])
            #kmeans = KMeans(n_clusters=groups_k[key], random_state=42)
            #kmeans_result = kmeans.fit_predict(senegal_df[X_features])
            #kmeans_labels = kmeans.labels_

            plt.figure(figsize=(10, 8))
            plt.scatter(pca_2d_result[:, 0], pca_2d_result[:, 1], c=senegal_df[target], cmap='viridis', alpha=0.6)
            plt.title(f"PCA of {key}, and then coloring by {target}")
            plt.xlabel("Principal Component 1")
            plt.ylabel("Principal Component 2")
            name = parse_feature_metadata(target)['name']
            plt.colorbar(label=f"{name}")
            plt.grid(True)
            plt.show()
            
            #for target in target_features:
                
            #    print_cluster_info_for_target(senegal_df, target, kmeans_labels)

            plot_cumulative_explained_variance_for_group_for_pca(senegal_df, key, X_features)

SENEGAL


In [946]:
from sklearn.decomposition import KernelPCA

In [947]:
def plot_cumulative_explained_variance_for_group_for_kpca(df, group_name, group_features):
    num_of_components = min(len(group_features),10)
    
    # Perform PCA with additional components
    kpca_full = KernelPCA(
                            n_components=num_of_components,     # how many principal axes you want
                            kernel="rbf",       # "rbf", "poly", "sigmoid", "cosine", etc.
                            gamma=0.1,          # only for RBF / polynomial kernels
                            fit_inverse_transform=False,  # True if you want to map back to X-space
                            random_state=0
                        )
    kpca_full_result = kpca_full.fit_transform(df[group_features])
    pca_on_kpca = PCA(n_components=num_of_components).fit(kpca_full_result)
    
    
    # Cumulative explained variance
    cumulative_variance = pca_on_kpca.explained_variance_ratio_.cumsum()
    
    # Plot cumulative explained variance
    plt.figure(figsize=(10, 6))
    plt.plot(range(1, len(cumulative_variance) + 1), cumulative_variance, marker='o')
    plt.title("Cumulative Explained Variance by PCA on KPCA Components")
    plt.xlabel("Number of Principal Components")
    plt.ylabel("Cumulative Explained Variance")
    plt.grid(True)
    plt.show()

In [948]:
print("NEPAL")
if PLOT_GRAPHS:
    for key, value in groups.items():
        print(f"PCA on {key} -> KPCA on PCA")
        X_features = convert_group_to_dummies(nepal_df, groups[key][0])

        # 2. Fit KernelPCA
        kpca_2d = KernelPCA(
            n_components=2,     # how many principal axes you want
            kernel="rbf",       # "rbf", "poly", "sigmoid", "cosine", etc.
            gamma=0.1,          # only for RBF / polynomial kernels
            fit_inverse_transform=False,  # True if you want to map back to X-space
            random_state=0
        )
        kpca_2d_result = kpca_2d.fit_transform(nepal_df[X_features])
        #kmeans = KMeans(n_clusters=groups_k[key], random_state=42)
        #kmeans_result = kmeans.fit_predict(nepal_df[X_features])
        #kmeans_labels = kmeans.labels_

        plt.figure(figsize=(10, 8))
        plt.scatter(kpca_2d_result[:, 0], kpca_2d_result[:, 1], cmap='viridis', alpha=0.6)
        plt.title(f"KPCA Visualization on {key}")
        plt.xlabel("Principal Component 1")
        plt.ylabel("Principal Component 2")
        plt.grid(True)
        plt.show()
        
        #for target in target_features:
            
        #    print_cluster_info_for_target(nepal_df, target, kmeans_labels)

        plot_cumulative_explained_variance_for_group_for_kpca(nepal_df, key, X_features)

NEPAL


In [949]:
if PLOT_GRAPHS:
    for key, value in groups.items():
        if 'Demographic' in key or 'Self-control' in key:
            continue
        
        print(f"PCA on {key} -> KPCA on PCA")
        X_features = convert_group_to_dummies(senegal_df, groups[key][1])

        # 2. Fit KernelPCA
        kpca_2d = KernelPCA(
            n_components=2,     # how many principal axes you want
            kernel="rbf",       # "rbf", "poly", "sigmoid", "cosine", etc.
            gamma=0.1,          # only for RBF / polynomial kernels
            fit_inverse_transform=False,  # True if you want to map back to X-space
            random_state=0
        )
        kpca_2d_result = kpca_2d.fit_transform(senegal_df[X_features])
        #kmeans = KMeans(n_clusters=groups_k[key], random_state=42)
        #kmeans_result = kmeans.fit_predict(senegal_df[X_features])
        #kmeans_labels = kmeans.labels_

        plt.figure(figsize=(10, 8))
        plt.scatter(kpca_2d_result[:, 0], kpca_2d_result[:, 1], cmap='viridis', alpha=0.6)
        plt.title(f"KPCA Visualization on {key}")
        plt.xlabel("Principal Component 1")
        plt.ylabel("Principal Component 2")
        plt.colorbar(label="Cluster")
        plt.grid(True)
        plt.show()
        
        #for target in target_features:
            
        #    print_cluster_info_for_target(senegal_df, target, kmeans_labels)

        plot_cumulative_explained_variance_for_group_for_kpca(senegal_df, key, X_features)

# Clustering Analysis

In [950]:
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import silhouette_score

In [951]:
def print_cluster_info_for_target(df, target_feature, kmeans_labels):
    target_name = parse_feature_metadata(target_feature)['name']
    for cluster_id in np.unique(kmeans_labels):
        indices = np.where(kmeans_labels == cluster_id)
        avg_label = np.mean(np.array(df[target_feature])[indices])
        print(f"Cluster {cluster_id}:\nAverage {target_name} = {avg_label:.2f}\n")

In [952]:
def plot_k_inertias(df, group_features):
    inertias = []
    K = range(1,11)
    X_scaled = StandardScaler().fit_transform(df[group_features])
    
    for k in K:
        km = KMeans(n_clusters=k, random_state=0).fit(X_scaled)
        inertias.append(km.inertia_)
    
    plt.plot(K, inertias, '-o')
    plt.xlabel('k'); plt.ylabel('Inertia')
    plt.title('Elbow Method for Optimal k')
    plt.show()

In [953]:
def plot_k_sillouhettes(df, group_features):
    scores = []
    X_scaled = StandardScaler().fit_transform(df[group_features])
    
    for k in range(2,11):
        km = KMeans(n_clusters=k, random_state=0).fit(X_scaled)
        labels = km.labels_
        scores.append(silhouette_score(X_scaled, labels))
    
    plt.plot(range(2,11), scores, '-o')
    plt.xlabel('k'); plt.ylabel('Avg. Silhouette Score')
    plt.title('Silhouette Method for Optimal k')
    plt.show()

In [954]:
if PLOT_GRAPHS:
    target_features = convert_group_to_dummies(senegal_df, target_cols)
    for key in groups.keys():
        if 'Self-control' in key:
            continue
        print(f"Group is: {key}")
        X_features = convert_group_to_dummies(senegal_df, groups[key][1])
        
        plot_k_inertias(senegal_df, X_features)
        
        plot_k_sillouhettes(senegal_df, X_features)
    
    

In [955]:
if PLOT_GRAPHS:
    target_features = convert_group_to_dummies(nepal_df, target_cols)
    for key, value in groups.items():
        print(f"Group is: {key}")
        X_features = convert_group_to_dummies(nepal_df, value[0])
        
        plot_k_inertias(nepal_df, X_features)
        
        plot_k_sillouhettes(nepal_df, X_features)
    
    

In [956]:
groups_k = {'Geographic Features': 2,
          'Demographic Features': 2,
          'Hope Features': 2,
          'Crop & Fertilizer Features': 3,
          'Plot Practice Features': 10,
          'Soil Analysis Faetures': 4,
          'Plow & Irrigation': 3,
          'Self-control Features': 2,
          'Household Features': 2}

In [957]:
# 2) Clustering functions
def cluster_kmeans(df, feature_cols, n_clusters=3, random_state=42):
    """
    Run K-Means on df[feature_cols].
    Returns: (labels array, fitted KMeans model)
    """
    X = df[feature_cols].to_numpy()
    # explicitly set n_init (old default was 10; new “auto” uses smarter init logic)
    km = KMeans(n_clusters=n_clusters,
                n_init="auto",          # ← explicitly match the old default
                random_state=random_state)
    labels = km.fit_predict(X)
    return labels, km

def cluster_agglomerative(df, feature_cols, n_clusters=3):
    """
    Run AgglomerativeClustering on df[feature_cols].
    Returns: (labels array, fitted AgglomerativeClustering model)
    """
    X = df[feature_cols].to_numpy()
    ag = AgglomerativeClustering(n_clusters=n_clusters)
    labels = ag.fit_predict(X)
    return labels, ag

# 3) Profiling function
def profile_clusters(df, labels, profiling_features):
    """
    Attach `labels` as df['cluster'], then return:
    - cluster sizes (Series)
    - mean of productivity_metrics per cluster (DataFrame)
    """
    n = len(profiling_features)
    ncols = int(math.ceil(math.sqrt(n)))
    nrows = int(math.ceil(n / ncols))
    
    fig, axes = plt.subplots(nrows, ncols,
                             figsize=(ncols * 4, nrows * 3),
                             squeeze=False)
    axes = axes.flatten()

    
    tmp = df.copy()
    tmp['cluster'] = labels

    for i, feat in enumerate(profiling_features):
        ax = axes[i]
        sns.boxplot(x=tmp['cluster'].astype(str), y=tmp[feat], ax=ax)
        
        orig_labels = [lbl.get_text() for lbl in ax.get_xticklabels()]
        wrapped_labels = ["\n".join([lab[i:i+30] for i in range(0, len(lab), 30)]) for lab in orig_labels]
        
        pos = np.arange(len(wrapped_labels))
        ax.set_xticks(pos)
        ax.set_xticklabels(wrapped_labels, rotation=45,
                        ha='right', fontsize=8)

        # per-axis label
        wrapped_label = "\n".join([feat[i:i+30] for i in range(0, len(feat), 30)])
        ax.set_xlabel(wrapped_label, fontsize=8, labelpad=8)
        ax.set_ylabel("")

    # turn off leftover axes
    for ax in axes[n:]:
        ax.set_visible(False)

    # tidy and title
    plt.tight_layout()
    plt.show()


In [958]:
def cluster_and_profile(df, country_key, group_name, profiling_feats, cluster_fn):
    
    cols = groups[group_name][0 if country_key == 'nepal' else 1]
    if not cols:
        print(f"Clustering group {group_name} doesnt exist in this country")
        return  # do nothing for groups missing in this country
    cols = convert_group_to_dummies(df, cols)
    labels, model = cluster_fn(df, cols, n_clusters=3)
    print(f"\nClustering by: {group_name}")
    profile_clusters(df, labels, profiling_feats)

In [959]:
import warnings

# scikit‐learn KMeans n_init warning
warnings.filterwarnings(
    "ignore",
    message=".*KMeans is known to have a memory leak.*",
    module="sklearn.cluster._kmeans"
)

# matplotlib FixedFormatter warning
warnings.filterwarnings(
    "ignore",
    message="FixedFormatter should only be used together with FixedLocator"
)

In [960]:
if PLOT_GRAPHS:
    for country_key, df in [('nepal', nepal_df), ('senegal', senegal_df)]:
        target_features = convert_group_to_dummies(df, target_cols)
        print(f"\n===== {country_key.upper()} =====")
        for method_name, cluster_fn in [
                ('K-Means', cluster_kmeans),
                ('Agglomerative', cluster_agglomerative)
            ]:
            print(f"\n--- {method_name} Clustering by Feature‐Group ---")
            for group_name in groups.keys():
                cluster_and_profile(df, country_key, group_name, target_features, cluster_fn)
